# CREATING version ID differences table and CYS LYS position failure analysis

## [next markdown after DEPvsINDEP mapping] 
> use v97 ENSP that map to UniProt ID from 5,361 and pull out .fasta file sequences matching ENSP from all versions [v85,v92,v94,v96,v97] to generate CYS/LYS position correct score

- depending on TODO1 i will know what is best source of ENSP to UKB to highlight stable ID mapping errors
### more details-- 
1. Version ID differences in dataset of labeled entries, IDs pulled across ENSMEBL releases (SOURCE: xref pipeline files) & 

2. pt2 is correctness score of CYS/LYS positions across ENSEMBL peptide fasta (SOURCE KEY uniprot--ensp is biomart overlap v97) 

### [initial attempt at version table findings] 
- previously found that in v97: only 10 ENSPversion IDs were different between GRCh37 and GRCh38
- saved these in BIOMART folder


### GOAL: 
**ANSWER how STABLE ID mapping fails in the context of targeted Cysteines and Lysines on 3,991 UniProt canonical sequences from 2018.  SHOW how time/dates of release look in terms of CORRECTNESS score correlation.**

8.28.19

9.5.19

mfpfox




In [1]:
# packages
import os
import sys
import numpy as np
import pandas as pd
import csv
import argparse
import Bio
from Bio import SeqIO
from ast import literal_eval # for mismap_score func
from statistics import mean
import jellyfish
import textdistance

In [2]:
sys.path.append("/Users/mariapalafox/Desktop/Toolbox/")

In [3]:
from all_funx import *

In [4]:
from IPython.display import display
pd.set_option('display.max_columns', None)
pd.options.display.max_seq_items = 2000

In [3]:
# TODO 0 
os.chdir("/Users/mariapalafox/Box Sync/CODE_DATA/dir_MAPpaper/BIOMART_ENSEMBL/")
print(os.listdir())

['v97sameStableIDGRCh37and38_diffENSPv_10.csv', '.DS_Store', 'TF38_22203.csv', 'NOTES.txt', 'GRCh37vsGRCh38_ensembl.sh', 'biomartGRCh38p.12_v97_filterCCDSIDonly.txt', 'v97StableIDmathchedGRCh38_duplicateStableID_fromUniqueUKBcrossRef_14.csv', 'entryUniProtCCDSset_simple.txt', 'StableIDs_allMatched_btw_GRCh37_38_ensemblv97_22202.csv', 'mapping_37.txt', 'biomart_CCDSIDonly_GRCh37_v97_peptideFasta.txt', 'StableIDs_ENST_ENSG_ENSP_v97ensembl_GRCh37_matchGRCh38_22193_fromBiomartCCDSonlyfilter.csv', 'UKBccdsEntry2stableENSP_fromTF38_stableIDkeyMatched_15745entries_counts2ENSPmultimapping.csv', 'TF37_22205.csv', 'Homo_sapiens.GRCh38.pep.all.fa', 'mapping_38.txt', 'v97versionDifferences_withSameStableID_GRCh37and38_22193.csv', 'biomartGRCh37p.13_v97_filterCCDSIDonly.txt', 'biomartGRCh37p.13_v97_filterHAVANAensemblTranscriptMatchOnly.txt', 'v97StableIDmathchedGRCh37_duplicateStableID_fromUniqueUKBcrossRef_18.csv']


In [5]:
# TODO 1
os.chdir("/Users/mariapalafox/Box Sync/CODE_DATA/dir_MAPpaper/TSV_UNIPROT_xref/CYSLYS_failure2map/")
print(os.listdir())

['interestingExampleProtein', 'Cys_ever_labeled_set_6315.csv', '.DS_Store', 'MISMAP_SCORE_CYS_LYS_14120pos_3636entries.csv', 'CYSLYS_everlabeled_groupby_entry_dictOfPositions_TODO0_3991.csv', 'CYS_LYS_EverLabeled_gene_valueCounts_3991.csv', 'v85_37_ENSP_fasta.csv', 'QC_v97_37n38_IDENTICALstableIDs_5361', 'Uniprot2ENSP_key4versioncheck_CYSLYSmapping_failure_3636unique_entry_fromv97assemblyOVERLAP.csv', 'countsfromfastaCCDS_filter_counts_all_AA.csv', 'v94_38_ENSP_fasta.csv', 'fastaTest.fa', 'v92_38_ENSP_fasta.csv', 'Lys_ever_labeled_set_9162.csv', 'v97_38_ENSP_fasta.csv', 'INSIGHTS_xref_100_is_only_for_canonical_never_written_explicitly_tho.txt', 'CYSLYS_everlabeled_groupby_entry_dictOfPositions_TODO0_3991 copy.csv', 'v96_38_ENSP_fasta.csv']


In [5]:
os.chdir("/Users/mariapalafox/Box Sync/CODE_DATA/dir_MAPpaper/TSV_UNIPROT_xref")
print(os.listdir())

['v96Homo_sapiens.GRCh38.pep.all.fa', 'v97Homo_sapiens.GRCh37.pep.all.fa', 'sharedv85_10272_uniqueENSP.csv', 'dynamic3_summaryMULTIMAPPING_ENSPid.csv', 'MISMAP_SCORED_differentNumUKBID_2124_v96_notTrue4myIdentityScore_4623.csv', 'Homo_sapiens.GRCh38.97.uniprot.tsv', 'MISMAP_SCORED_differentNumUKBID_3885_v97_True4myIdentityScore_6052.csv', 'uniprotIDs3979.csv', 'groupedMISMAP_score_85_FALSEidentity_1805.csv', 'v85_labeledEverUKBID_xref100_sourceref100_15.csv', 'MISMAP_SCORE_xref_SharedID_allReleases_3979_UKBkey.csv', '.DS_Store', 'MISMAP_SCORED_differentNumUKBID_3878_v92_True4myIdentityScore_5957.csv', 'MISMAP_SCORED_v94_ENSP_posDict_checked_10699.csv', 'v94_labeledEverUKBID_xref100_sourceref100_6097.csv', 'MISMAP_SCORED_v97_ENSP_posDict_checked_10650.csv', 'v85_labeledEverUKBID_xref100_sourceref100_14.csv', 'MISMAP_SCORED_described_10750.csv', 'MISMAP_SCORED_v96_ENSP_posDict_checked_10750.csv', 'Homo_sapiens.GRCh38.92.uniprot.tsv', 'v96_labeledEverUKBID_xref100_sourceref100_6152.csv', 

### histogram of ever labeled entries, to show distribution of times detected 
- file input : CYS_LYS_EverLabeled_gene_valueCounts_3991.csv

# positions of CYS and LYS
- "Cys_ever_labeled_set_6315.csv"
- "Lys_ever_labeled_set_9162.csv"

# xref files from ENSEMBL - GRCh37(85) and GRCh38(92, 94, 96, 97)
#### VEP85  released 7/21/16
#### VEP92 released 3/7/18
#### VEP94 released 9/5/18
#### VEP96 release 4/3/19
### files below: 
- all3991_everLabeled_entries_simple.csv
- Homo_sapiens.GRCh37.85.uniprot.tsv
- Homo_sapiens.GRCh38.92.uniprot.tsv
- Homo_sapiens.GRCh38.94.uniprot.tsv
- Homo_sapiens.GRCh38.96.uniprot.tsv
- Homo_sapiens.GRCh38.97.uniprot.tsv

### xref tsv files have mapping to SOURCE = UniProt/RefSeq/PDB
* contains Homo_sapiens.GRCh37.85.uniprot.tsv mapping
* why is this ensembl v85??
    * they stopped doing mapping for ukb after 85 release for reference 37
    
### README_uniprot.tsv:
The files provided here are to give highlevel summaries of datasets in an easy to parse format. All TSV files are gzipped and their first line is a header line for file content.

+++++++++++++++++++++++++ Stable ID to uniprot Ac +++++++++++++++++++++++++

Provides mappings from Gene, Transcript and Translation stable identifiers to uniprot accessions with reports as to the % identity of the hit where applicable. Dumps contain all Ensembl exeternal database names which started with UniProt so duplication of hits is possible.

File are named Species.assembly.release.uniprot.tsv.gz


The 38 version is from vep96 release The 37 version is from vep85, they don't have a uniprot version past 85 ftp://ftp.ensembl.org/pub/grch37/release-96/tsv/homo_sapiens/

---

In [49]:
labeled = "all3991_everLabeled_entries_simple.csv"
v85 = "Homo_sapiens.GRCh37.85.uniprot.tsv"
v92 = "Homo_sapiens.GRCh38.92.uniprot.tsv"
v94 = "Homo_sapiens.GRCh38.94.uniprot.tsv"
v96 = "Homo_sapiens.GRCh38.96.uniprot.tsv"
v97 = "Homo_sapiens.GRCh38.97.uniprot.tsv"

In [50]:
labeledever = pd.read_csv(labeled)
v85 = pd.read_table(v85)
v92 = pd.read_table(v92)
v94 = pd.read_table(v94)
v96 = pd.read_table(v96)
v97 = pd.read_table(v97)

# TABLE of xref file 100% identity score comparison between releases
    - later is code for my version of TABLE, doing identity score calculation using canonical uniprot sequences

## FILTER-1: keep only rows with 'db_name' == 'Uniprot/SWISSPROT'

In [51]:
def dropnonswiss(df):
    df2 = df[df['db_name']=="Uniprot/SWISSPROT"]
    return df2
# dumping all non swissprot entries

In [52]:
v85 = dropnonswiss(v85)
v92 = dropnonswiss(v92)
v94 = dropnonswiss(v94)
v96 = dropnonswiss(v96)
v97 = dropnonswiss(v97)

In [53]:
Lever = list(labeledever['Entry'])
print(len(Lever))

3991


## FILTER-2: 'xref' values must be uniprot entry also contained in LabeledEVER set of 3991

In [54]:
def dropNotLabeled(df,Lever):
    df['Labeled_UKB_ID'] = np.where(df['xref'].isin(Lever),"True","False")
    df2 = df[df['Labeled_UKB_ID']=="True"]
    df2.sort_values(by=['xref'],inplace=True)
    return df2

In [55]:
L85 = dropNotLabeled(v85,Lever)
L92 = dropNotLabeled(v92,Lever)
L94 = dropNotLabeled(v94,Lever)
L96 = dropNotLabeled(v96,Lever)
L97 = dropNotLabeled(v97,Lever)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [56]:
print(L85.shape)
print(L92.shape)
print(L94.shape)
print(L96.shape)
print(L97.shape)

(10280, 10)
(10490, 10)
(10718, 10)
(10769, 10)
(10669, 10)


In [57]:
# how many unique UNIPROT IDs in this file???
def uniqueCount(df, colname):
    print(len(df[colname].unique()))

In [58]:
uniqueCount(L85,'xref') #3984
uniqueCount(L92,'xref') #3985
uniqueCount(L94,'xref') #3990
uniqueCount(L96,'xref') #3990
uniqueCount(L97,'xref') #3990

3984
3985
3990
3990
3990


# saving xref v85 set of entries 3984

In [59]:
def diffList(L1, L2):
    return list(L1 - L2)

In [60]:
uni85 = set(L85['xref'].tolist())
uni92 = set(L92['xref'].tolist())
uni94 = set(L94['xref'].tolist())
uni96 = set(L96['xref'].tolist())
uni97 = set(L97['xref'].tolist())

diffList(uni97,uni92)
diffList(uni97,uni85)
diffList(uni94,uni96)
diffList(uni94,uni97)

shared = uni85 & uni92 & uni94 & uni96 & uni97

print(len(set(shared)))
shared = list(shared)
# 3980
#sh = pd.Series(shared)
#sh.to_csv("MISMAP_SCORE_xref_Swiss_labeledTrue_SharedIDs_allReleases_3980_UKBkey.csv")

3980


## FILTER-3: 'xref' values must be uniprot entry Labeled and SHARED between all ENSEMBL releases
- total 3980 shared

In [61]:
def dropNotLabeled(df,Lever):
    df['Shared_UKB_ID'] = np.where(df['xref'].isin(Lever),"True","False")
    df2 = df[df['Shared_UKB_ID']=="True"].copy()
    df2.sort_values(by=['xref'],inplace=True)
    return df2

In [62]:
L85 = dropNotLabeled(L85,shared)
L92 = dropNotLabeled(L92,shared)
L94 = dropNotLabeled(L94,shared)
L96 = dropNotLabeled(L96,shared)
L97 = dropNotLabeled(L97,shared)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [23]:
uniqueCount(L85,'xref') 
uniqueCount(L92,'xref') 
uniqueCount(L94,'xref') 
uniqueCount(L96,'xref') 
uniqueCount(L97,'xref') 
# all should == 3980

3980
3980
3980
3980
3980


In [63]:
print(L85.shape)
print(L92.shape)
print(L94.shape)
print(L96.shape)
print(L97.shape)

(10274, 11)
(10480, 11)
(10700, 11)
(10751, 11)
(10651, 11)


In [69]:
delme = 'ENSP00000368645'
L85[L85['protein_stable_id'] == delme]

,gene_stable_id,transcript_stable_id,protein_stable_id,xref,db_name,info_type,source_identity,xref_identity,linkage_type,Labeled_UKB_ID,Shared_UKB_ID
66096,ENSG00000168255,ENST00000379340,ENSP00000368645,Q9GZM3,Uniprot/SWISSPROT,SEQUENCE_MATCH,100,100,-,True,True
66097,ENSG00000168255,ENST00000379340,ENSP00000368645,Q9H1A7,Uniprot/SWISSPROT,DIRECT,-,-,-,True,True


In [74]:
delme = 'ENSP00000368645'
L92[L92['protein_stable_id'] == delme]

,gene_stable_id,transcript_stable_id,protein_stable_id,xref,db_name,info_type,source_identity,xref_identity,linkage_type,Labeled_UKB_ID,Shared_UKB_ID
37782,ENSG00000168255,ENST00000379340,ENSP00000368645,Q9H1A7,Uniprot/SWISSPROT,DIRECT,99,99,-,True,True


In [75]:
# removing duplicate ENSP found later in markdown when generating KEYS
delme = 'ENSP00000368645'
L85 = L85[L85['protein_stable_id'] != delme].copy()
L92 = L92[L92['protein_stable_id'] != delme].copy()
L94 = L94[L94['protein_stable_id'] != delme].copy()
L96 = L96[L96['protein_stable_id'] != delme].copy()
L97 = L97[L97['protein_stable_id'] != delme].copy()

In [152]:
print(L85.shape)
print(L92.shape)
print(L94.shape)
print(L96.shape)
print(L97.shape)

(10272, 11)
(10479, 11)
(10699, 11)
(10750, 11)
(10650, 11)


In [153]:
uniqueCount(L85,'xref') 
uniqueCount(L92,'xref') 
uniqueCount(L94,'xref') 
uniqueCount(L96,'xref') 
uniqueCount(L97,'xref') 
# all should == 3979

3979
3979
3979
3979
3979


In [78]:
uniqueCount(L85,'protein_stable_id') 
uniqueCount(L92,'protein_stable_id') 
uniqueCount(L94,'protein_stable_id') 
uniqueCount(L96,'protein_stable_id') 
uniqueCount(L97,'protein_stable_id') 


10272
10479
10699
10750
10650


In [79]:
L85.reset_index(drop=True,inplace=True)
L92.reset_index(drop=True,inplace=True)
L94.reset_index(drop=True,inplace=True)
L96.reset_index(drop=True,inplace=True)
L97.reset_index(drop=True,inplace=True)

In [80]:
L85.to_csv("sharedv85_10272_uniqueENSP.csv",index=False)
L92.to_csv("sharedv92_10479_uniqueENSP.csv",index=False)
L94.to_csv("sharedv94_10699_uniqueENSP.csv",index=False)
L96.to_csv("sharedv96_10750_uniqueENSP.csv",index=False)
L97.to_csv("sharedv97_10650_uniqueENSP.csv",index=False)

In [81]:
L85.head(2)

,gene_stable_id,transcript_stable_id,protein_stable_id,xref,db_name,info_type,source_identity,xref_identity,linkage_type,Labeled_UKB_ID,Shared_UKB_ID
0,ENSG00000033178,ENST00000322244,ENSP00000313454,A0AVT1,Uniprot/SWISSPROT,DIRECT,-,-,-,True,True
1,ENSG00000033178,ENST00000420827,ENSP00000399234,A0AVT1,Uniprot/SWISSPROT,DIRECT,-,-,-,True,True


In [82]:
L97.head(3)

,gene_stable_id,transcript_stable_id,protein_stable_id,xref,db_name,info_type,source_identity,xref_identity,linkage_type,Labeled_UKB_ID,Shared_UKB_ID
0,ENSG00000033178,ENST00000322244,ENSP00000313454,A0AVT1,Uniprot/SWISSPROT,DIRECT,100,100,-,True,True
1,ENSG00000033178,ENST00000420827,ENSP00000399234,A0AVT1,Uniprot/SWISSPROT,DIRECT,-,-,-,True,True
2,ENSG00000117868,ENST00000251527,ENSP00000251527,A0FGR8,Uniprot/SWISSPROT,DIRECT,-,-,-,True,True


## Release-specific UNIPROTID <----> ENSP KEYS
- use these keys for extracting rows from ENSEMBL release FASTA files, downstream is mismap scoring of LYS/CYS positions

## TODO-2: create key for each xref file that links UKB no isoform spec ID to multiple ENSP IDs {UKB entry (redundant) : ENSP associated}

In [83]:
key85 = L85[['protein_stable_id', 'xref']].copy()
key92 = L92[['protein_stable_id', 'xref']].copy()
key94 = L94[['protein_stable_id', 'xref']].copy()
key96 = L96[['protein_stable_id', 'xref']].copy()
key97 = L97[['protein_stable_id', 'xref']].copy()

In [35]:
# non redundant ENSP keys
key85.to_csv("key85_10272.csv",index=False)
key92.to_csv("key92_10479.csv",index=False)
key94.to_csv("key94_10699.csv",index=False)
key96.to_csv("key96_10750.csv",index=False)
key97.to_csv("key97_10650.csv",index=False)

# [Multimapping counts table using xref 100 score] using the xref files (SOURCE of mismap analysis stable ID keys later part of markdown) to count unique stable IDs
**filtered so far for**
1. swissprot xref
2. entry in LabeledEver set
3. entry shared with all other releases (3979)

In [36]:
def checkColumnValues(df, col):
    print(df[col].value_counts().reset_index().rename(columns={'index':col, col:'Count'}))

In [37]:
checkColumnValues(L85,"info_type")
print()
checkColumnValues(L92,"info_type")
print()
checkColumnValues(L94,"info_type")
print()
checkColumnValues(L96,"info_type")
print()
checkColumnValues(L97,"info_type")

        info_type  Count
0          DIRECT  10256
1  SEQUENCE_MATCH     16

        info_type  Count
0          DIRECT  10459
1  SEQUENCE_MATCH     20

        info_type  Count
0          DIRECT  10698
1  SEQUENCE_MATCH      1

  info_type  Count
0    DIRECT  10750

        info_type  Count
0          DIRECT  10648
1  SEQUENCE_MATCH      2


## FILTER (for version differences table) -4: filter for only 100% score in 'xref_identity' 

In [38]:
# dropping all rows where xref_identity (ukb sequence) was not matched 100 % to ensembl
def dropNot100xref_identity(df):
    df2 = df[df['xref_identity']=="100"]
    return df2

In [39]:
Lx85 = dropNot100xref_identity(L85)
Lx92 = dropNot100xref_identity(L92)
Lx94 = dropNot100xref_identity(L94)
Lx96 = dropNot100xref_identity(L96)
Lx97 = dropNot100xref_identity(L97)

In [40]:
print(Lx85.shape)
print(Lx92.shape)
print(Lx94.shape)
print(Lx96.shape)
print(Lx97.shape)

(14, 11)
(6076, 11)
(6229, 11)
(6264, 11)
(6187, 11)


## FILTER (for version differences table) -5: filter for only 100% score in 'source_identity' 

In [41]:
def dropNot100source_identity(df):
    df2 = df[df['source_identity']=="100"]
    return df2

In [42]:
Lx85 = dropNot100source_identity(Lx85)
Lx92 = dropNot100source_identity(Lx92)
Lx94 = dropNot100source_identity(Lx94)
Lx96 = dropNot100source_identity(Lx96)
Lx97 = dropNot100source_identity(Lx97)

In [182]:
print(Lx85.shape)
print(Lx92.shape)
print(Lx94.shape)
print(Lx96.shape)
print(Lx97.shape)

(14, 11)
(5943, 11)
(6097, 11)
(6136, 11)
(6064, 11)


In [44]:
print(len(Lx85['xref'].unique()))
print(len(Lx92['xref'].unique()))
print(len(Lx94['xref'].unique()))
print(len(Lx96['xref'].unique()))
print(len(Lx97['xref'].unique()))

10
3871
3895
3899
3897


In [45]:
print(len(Lx85['gene_stable_id'].unique()))
print(len(Lx92['gene_stable_id'].unique()))
print(len(Lx94['gene_stable_id'].unique()))
print(len(Lx96['gene_stable_id'].unique()))
print(len(Lx97['gene_stable_id'].unique()))

10
4207
4248
4244
4241


In [46]:
print(len(Lx85['transcript_stable_id'].unique()))
print(len(Lx92['transcript_stable_id'].unique()))
print(len(Lx94['transcript_stable_id'].unique()))
print(len(Lx96['transcript_stable_id'].unique()))
print(len(Lx97['transcript_stable_id'].unique()))

14
5943
6097
6136
6064


In [47]:
print(len(Lx85['protein_stable_id'].unique()))
print(len(Lx92['protein_stable_id'].unique()))
print(len(Lx94['protein_stable_id'].unique()))
print(len(Lx96['protein_stable_id'].unique()))
print(len(Lx97['protein_stable_id'].unique()))

14
5943
6097
6136
6064


In [183]:
# multi mapping and xref 100 table: compare this with my TRUE identity IDs
Lx85.to_csv("v85_labeledEverUKBID_xref100_sourceref100_14.csv")
Lx92.to_csv("v92_labeledEverUKBID_xref100_sourceref100_5943.csv")
Lx94.to_csv("v94_labeledEverUKBID_xref100_sourceref100_6097.csv")
Lx96.to_csv("v96_labeledEverUKBID_xref100_sourceref100_6136.csv")
Lx97.to_csv("v97_labeledEverUKBID_xref100_sourceref100_6064.csv")

---
---
---

# [Comparing xref identity score unique value counts to my identity score: Multimapping counts table using my 100 identity score]
### adding column if ENSP ID is 100% identical to my uniprot fasta file... (later part of markdown)

```python
# all ENSP that are TRUE for my identity score match canonical uniprot sequence
# all ENSP are unique, tested list == set(list)
# all list will be used to compare my identity score to xref score
v85TRUEkey = v85TRUE['ENSP'].tolist()
v92TRUEkey = v92TRUE['ENSP'].tolist()
v94TRUEkey = v94TRUE['ENSP'].tolist()
v96TRUEkey = v96TRUE['ENSP'].tolist()
v97TRUEkey = v97TRUE['ENSP'].tolist()
print(len(v85TRUEkey))
print(len(v92TRUEkey))
print(len(v94TRUEkey))
print(len(v96TRUEkey))
print(len(v97TRUEkey))
```

In [156]:
print(L85.shape)
print(L92.shape)
print(L94.shape)
print(L96.shape)
print(L97.shape)
print(len(v85TRUEkey))
print(len(v92TRUEkey))
print(len(v94TRUEkey))
print(len(v96TRUEkey))
print(len(v97TRUEkey))

(10272, 11)
(10479, 11)
(10699, 11)
(10750, 11)
(10650, 11)
5925
5957
6097
6127
6052


In [157]:
L85.columns

Index(['gene_stable_id', 'transcript_stable_id', 'protein_stable_id', 'xref',
       'db_name', 'info_type', 'source_identity', 'xref_identity',
       'linkage_type', 'Labeled_UKB_ID', 'Shared_UKB_ID'],
      dtype='object')

In [158]:
v85TRUEkey[:10]

['ENSP00000313454',
 'ENSP00000279907',
 'ENSP00000347532',
 'ENSP00000374565',
 'ENSP00000387051',
 'ENSP00000263383',
 'ENSP00000374592',
 'ENSP00000386616',
 'ENSP00000346627',
 'ENSP00000387361']

In [159]:
def dropNotLabeled(df,Lever):
    df['my_identity_100'] = np.where(df['protein_stable_id'].isin(Lever),"True","False")
    df2 = df[df['my_identity_100']=="True"].copy()
    df2.sort_values(by=['xref'],inplace=True)
    return df2

In [160]:
L85 = dropNotLabeled(L85,v85TRUEkey)
L92 = dropNotLabeled(L92,v92TRUEkey)
L94 = dropNotLabeled(L94,v94TRUEkey)
L96 = dropNotLabeled(L96,v96TRUEkey)
L97 = dropNotLabeled(L97,v97TRUEkey)

In [161]:
print(L85.shape)
print(L92.shape)
print(L94.shape)
print(L96.shape)
print(L97.shape)

(5925, 12)
(5957, 12)
(6097, 12)
(6127, 12)
(6052, 12)


In [162]:
print(len(L85['xref'].unique()))
print(len(L92['xref'].unique()))
print(len(L94['xref'].unique()))
print(len(L96['xref'].unique()))
print(len(L97['xref'].unique()))

3884
3878
3895
3890
3885


In [163]:
print(len(L85['gene_stable_id'].unique()))
print(len(L92['gene_stable_id'].unique()))
print(len(L94['gene_stable_id'].unique()))
print(len(L96['gene_stable_id'].unique()))
print(len(L97['gene_stable_id'].unique()))

4241
4215
4248
4235
4229


In [164]:
print(len(L85['transcript_stable_id'].unique()))
print(len(L92['transcript_stable_id'].unique()))
print(len(L94['transcript_stable_id'].unique()))
print(len(L96['transcript_stable_id'].unique()))
print(len(L97['transcript_stable_id'].unique()))

5925
5957
6097
6127
6052


In [165]:
print(len(L85['protein_stable_id'].unique()))
print(len(L92['protein_stable_id'].unique()))
print(len(L94['protein_stable_id'].unique()))
print(len(L96['protein_stable_id'].unique()))
print(len(L97['protein_stable_id'].unique()))

5925
5957
6097
6127
6052


In [ ]:
# variables made later in markdown, col from these df made into list to filter for above counts
# v85TRUE.to_csv("v85_labeledBefore_myidentity100_xref_5925.csv",index=False)
# v92TRUE.to_csv("v92_labeledBefore_myidentity100_xref_5957.csv",index=False)
# v94TRUE.to_csv("v94_labeledBefore_myidentity100_xref_6097.csv",index=False)
# v96TRUE.to_csv("v96_labeledBefore_myidentity100_xref_6127.csv",index=False)
# v97TRUE.to_csv("v97_labeledBefore_myidentity100_xref_6052.csv",index=False)

# these files are the fasta file only the rows with ENSP that was identical to canonical uniprot seq....
v85_Ensembl_fasta_identical_canonicalUKB_5925.csv
v92_Ensembl_fasta_identical_canonicalUKB_5957.csv
v94_Ensembl_fasta_identical_canonicalUKB_6097.csv
v96_Ensembl_fasta_identical_canonicalUKB_6127.csv
v97_Ensembl_fasta_identical_canonicalUKB_6052.csv

---
---
---

---
---
---

---
---
---

# MISMAP SCORE with CYS/LYS positions

In [7]:
os.chdir("/Users/mariapalafox/Box Sync/CODE_DATA/dir_MAPpaper/TSV_UNIPROT_xref/CYSLYS_failure2map")
print(os.listdir())

['v96Homo_sapiens.GRCh38.pep.all.fa', 'Cys_ever_labeled_set_6315.csv', 'v85_labeledEverUKBID_xref100_sourceref100_15.csv', 'v96_labeledEverUKBID_xref100_sourceref100_6152.csv', 'CYS_LYS_EverLabeled_gene_valueCounts_3991.csv', 'v97Homo_sapiens.GRCh38.pep.all.fa', 'v92_labeledEverUKBID_xref100_sourceref100_5951.csv', 'countsfromfastaCCDS_filter_counts_all_AA.csv', 'v97_labeledEverUKBID_xref100_sourceref100_6080.csv', 'v94Homo_sapiens.GRCh38.pep.all.fa', 'v92Homo_sapiens.GRCh38.pep.all.fa', 'v94_labeledEverUKBID_xref100_sourceref100_6112.csv', 'Lys_ever_labeled_set_9162.csv', 'v85Homo_sapiens.GRCh37.pep.all.fa']


## TODO-0: create dictionary of {everLabeled entry : [list of CYS and LYS positions labeled]}
- GOAL: need to check if cys lys positions are matched from ensembl peptide fasta seqs to labeled position sequences (2012 to 2018 project out files)
    - then i will to find all CYS LYS positions in peptides from ensembl and see if there is overlap with the everLabeled CYS and LYS positions 
    - % correct score for each entry in each release
    - % correct score for each release
    
```bash
# example of Cys_ever_labeled_set_6315.csv file
Ever_labeled_Cys_set,subset,entry,pos
Q68E01_C596,Cysteine,Q68E01,C596
P30101_C57,Cysteine,P30101,C57
Q9UBQ7_C216,Cysteine,Q9UBQ7,C216
```

```bash
# way to check if dictionary of ID:labeled list is correct, list length should match this file labeled_counts value
Entry,labeled_counts
Q09666,112
P21333,75
O75369,65
Q15149,58
P35579,56
```

### code 
```python 

# import files
cysever = "Cys_ever_labeled_set_6315.csv"
lysever = "Lys_ever_labeled_set_9162.csv"
cysever = pd.read_csv(cysever)
lysever = pd.read_csv(lysever)

# drop columns that i dont need
cys = cysever.filter(['entry','pos'],axis=1)
lys = lysever.filter(['entry','pos'],axis=1)

# appending together at bottom, total shape is 15,477
appendedcyslys = cys.append(lys,ignore_index=True)
appendedcyslys.shape

# creating list of positions labeled for each entry
groupedCYSLYS = appendedcyslys.groupby('entry')['pos'].apply(list)
groupedCYSLYS2 = pd.DataFrame(groupedCYSLYS,index=None)
groupedCYSLYS2.reset_index(inplace=True)

# shape of df now == # unique entries, 3991

###### result so far #######
entry	pos
0	A0AVT1	[C625, C433, C721, C347, C546, C770, K409, K10...
1	A0FGR8	[C181]
2	A0JNW5	[K645]
3	A0MZ66	[C565]
4	A1KXE4	[C63]
################################

# creating column that is # of times entry has been labeled
groupedCYSLYS2['labeled_pos_count'] = groupedCYSLYS2['pos'].str.len()

###### result so far #######
entry	pos	labeled_pos_count
0	A0AVT1	[C625, C433, C721, C347, C546, C770, K409, K10...	11
1	A0FGR8	[C181]	1
2	A0JNW5	[K645]	1
################################

# sorting df by decending # of times entry has been labeled
groupedCYSLYS2_sort = groupedCYSLYS2.sort_values('labeled_pos_count',ascending=False)
groupedCYSLYS2_sort.reset_index(inplace=True,drop=True)

###### result so far #######
entry	pos	labeled_pos_count
0	Q09666	[C2162, C1833, C2806, C5502, C1967, C108, C190...	112
1	P21333	[C2199, C2293, C2160, C574, C1157, C2378, C116...	75
2	O75369	[C2556, C1280, C991, C1158, C660, C1876, C183,...	65
################################


# funx that turns list of labeled positions into sorted dictionary
# each key is index+1 of cys or lys that was labeled
# each value is chr of C or K
def get_pos_dictionary(pdcol):
    list_of_dicts = []
    
    for row in pdcol.tolist():
        listofkey=[]
        listofvalues=[]
        
        for i in row:
            v = i[0]
            listofvalues.append(v)
            k = i[1:]
            listofkey.append(k)
        di = dict(zip(listofkey, listofvalues))
        intdict = {int(oldkey) : val for oldkey, val in di.items()}
        sdi = dict(sorted(intdict.items(),reverse=False))
        list_of_dicts.append(sdi)
    return list_of_dicts


# calling funx by passing in df series (pos column)
list_of_dicts = get_pos_dictionary(groupedCYSLYS2_sort['pos'])

# turning list of dictionaries into a pandas column
dictSeries = pd.Series(list_of_dicts)

# adding dict col to original dataframe
groupedCYSLYS2_sort['pos_dict'] = dictSeries

###### result so far #######
entry	pos	labeled_pos_count	pos_dict
0	Q09666	[C2162, C1833, C2806, C5502, C1967, C108, C190...	112	{85: 'K', 108: 'C', 128: 'K', 243: 'K', 252: '...
1	P21333	[C2199, C2293, C2160, C574, C1157, C2378, C116...	75	{53: 'C', 58: 'K', 87: 'K', 210: 'C', 220: 'K'...
2	O75369	[C2556, C1280, C991, C1158, C660, C1876, C183,...	65	{15: 'K', 16: 'K', 26: 'C', 60: 'K', 93: 'K', ...
3	Q15149	[C1136, C4071, C4574, C888, C317, C545, C992, ...	58	{191: 'K', 236: 'K', 253: 'K', 317: 'C', 338: ...
################################

# saving new file with positions of labeled cys and lys in dictionary form
groupedCYSLYS2_sort.to_csv("CYSLYS_everlabeled_groupby_entry_dictOfPositions_TODO0_3991.csv")
                                                                   
```

----
----
-----

## TODO-1: process Ensembl fasta peptide files into csv with ENSP col (code below) and Uniprot ID column (added using release specific KEYS) filter out ENSP IDs NOT contained in each release KEY {ENSP: UNIPROT ID}
```python
TODO 1

# script from tool box for working with fasta files
# from processUniprotFasta.py
"""
UPDATES:
- added gencode or uniprot or refseq flag option depending on source of fasta
    7.24.19
    7.29.19
"""
# import argparse
# import bio
# #from Bio import SeqIO
pep85 = "v85Homo_sapiens.GRCh37.pep.all.fa"
pep92 = "v92Homo_sapiens.GRCh38.pep.all.fa"
pep94 = "v94Homo_sapiens.GRCh38.pep.all.fa"
pep96 = "v96Homo_sapiens.GRCh38.pep.all.fa"
pep97 = "v97Homo_sapiens.GRCh38.pep.all.fa"
test = "fastaTest.fa"
#
#
0	>ENSP00000488240.1 
1	pep 
2	chromosome:GRCh38:CHR_HSCHR7_2_CTG6:142847306:142847317:1 
3   gene:ENSG00000282253.1 
4	transcript:ENST00000631435.1 
5	gene_biotype:TR_D_gene 
6	transcript_biotype:TR_D_gene gene_symbol:TRBD1 description:T cell receptor beta 
7	diversity 
8	1 
9	[Source:HGNC Symbol;Acc:HGNC:12158]
	GGGGGGGGGGGGG
#
#
#
def fastaToDF(filename,dbtype): 
    # parse sequence fasta file
    identifiers = [seq_record.id for seq_record in SeqIO.parse(filename, "fasta")]
    lengths = [len(seq_record.seq) for seq_record in SeqIO.parse(filename, "fasta")]
    proSeq = [seq_record.seq for seq_record in SeqIO.parse(filename, "fasta")]
    print(identifiers)
    print(lengths)
    print(proSeq)
fastaToDF(test,"x")

def fastaToDF(filename):  
    # parse sequence fasta file
    identifiers = [seq_record.id for seq_record in SeqIO.parse(filename, "fasta")]
    lengths = [len(seq_record.seq) for seq_record in SeqIO.parse(filename, "fasta")]
    proSeq = [seq_record.seq for seq_record in SeqIO.parse(filename, "fasta")]
#     print(identifiers)
#     print(lengths)
#     print(proSeq)
    ensp = []
    enspv = []
    proseq = []
    for id in identifiers:
        splitID = id.split('.')
        stable = splitID[0]
        ensp.append(stable)
        enspv.append(id)
    for ps in proSeq:
        s = str(ps)
        proseq.append(s)
    #converting lists to pandas Series
    s1 = pd.Series(enspv, name='ENSPv')
    s2 = pd.Series(ensp, name= 'ENSP')
    s3 = pd.Series(lengths, name='Length')
    s4 = pd.Series(proseq, name='proSequence')
    series = [s1,s2,s3,s4]
    df = pd.concat(series, axis=1)

    return(df)

# call funx
v85 = fastaToDF(pep85)
v92 = fastaToDF(pep92)
v94 = fastaToDF(pep94)
v96 = fastaToDF(pep96)
v97 = fastaToDF(pep97)

def h(df):
    print(df.head(5))

h(v85)
h(v92)
h(v94)
h(v96)
h(v97)
v85.to_csv("v85_37_ENSP_fasta.csv",index=False)
v92.to_csv("v92_38_ENSP_fasta.csv",index=False)
v94.to_csv("v94_38_ENSP_fasta.csv",index=False)
v96.to_csv("v96_38_ENSP_fasta.csv",index=False)
v97.to_csv("v97_38_ENSP_fasta.csv",index=False)
```



---

---
---
---

---
---
---

### ANOTHER SOURCE OF KEY ENSP--UNIPROT is UniProt ID mapping file, which includes uniprot isoform details and ENSG, ENST, and ENSP mapping to Ensembl (part of McGarvey paper)
> Not good idea to use in mismap of CYS LYS analysis becuase its isoform specific and the GOAL is to show stable ID failure

##### file made from UNIPROT ID mapping file (isoform uniprot style mapping to stable ensembl)
> isoCount.to_csv("UniprotIDMAPPING_isoformCount__labeledBefore_ENSPsource_3955unique entries.csv")
> ensp_labeled.to_csv("UniprotIDMAPPING_ENSPsourcetypefiltered_labeledBeforeEntry_7601")
> trueunilabeled.to_csv("UniprotIDMAPPING_trueunilabeled_incl_ENST_ENSP_19157.csv")

---
---
---

## TODO-2: created key for each xref file that links 3979 UKB IDs to multiple ENSP IDs 
- files created above during xref release file filtering

## TODO-3: filter UKBccds fasta downloaded 2018 to contain only 3979 UKB IDs also in the xref file....SOURCE of mapping KEY 
```bash
# simple file of 3,991 ever labeled entries in UKBccds set
ukbccds = "all3991_everLabeled_entries_simple.csv"
myukb = "countsfromfastaCCDS_filter_counts_all_AA.csv"
```

## TODO-4: merge TODO-2 & TODO-3 on UKB ID
- GOAL: create df with UKB ID everlabeled / sequence of UKBccds protein / ENSP ID linked to UKBccds ID
    - there will be multi mapping since the UKB ID is associated with several ENSP IDs

## TODO-5: merge TODO-4 with the TODO-1 file containing ENSP protein sequences
- GOAL: redundant UKB IDs and peptide sequences combined with nonredundant stable ENSP linked to that ID and their peptide sequences

---
---
---

---
---
---

## TODO-1: turn Ensembl fasta peptide files into csv (done) and parse versioned ID into an equivalent stable ID columns 

http://biopython.org/DIST/docs/tutorial/Tutorial.html#htoc18


# TODO-2 (post filtering KEYS for redundancy = 3979 Uniprot IDs) .. filtering the uniprot dictionary of CYS and LYS positions '2018'
#### GOAL = how many targetable amino acids do i know have to work with in mis mapping analysis??

In [18]:
def addcolumnconditional(mapList, df, dfcol, newcol): 
    mendel = []
    for g in df[dfcol]:
        if g in mapList: 
            mendel.append("True")
        else:
            mendel.append("False")
    df.loc[:,newcol] = mendel
#     df.drop(df[df[newcol] == "False"].index, inplace = True)
#     df.reset_index(inplace=True, drop=True)
    return df

In [198]:
targets= "/Users/mariapalafox/Box Sync/CODE_DATA/dir_MAPpaper/TSV_UNIPROT_xref/CYSLYS_failure2map/CYSLYS_everlabeled_groupby_entry_dictOfPositions_TODO0_3991.csv"
tar= pd.read_csv(targets)
describeMe(tar)

(3991, 3)
Index(['entry', 'labeled_pos_count', 'pos_dict'], dtype='object')
entry                0
labeled_pos_count    0
pos_dict             0
dtype: int64


In [6]:
key85 = pd.read_csv("key85_10272.csv")
ls85 = set(key85['xref'].tolist())
ls85 = list(ls85)

In [97]:
print(len(ls85))

3979


In [98]:
# for 3979 uniprot IDs shared in all releases
xrefdict = addcolumnconditional(ls85, tar, 'entry','in3979xref')

# only uniprot IDs shared in all release files that have FALSE for identical to canonical sequence

In [199]:
# for 1805 uniprot IDs shared in all releases rows with FALSE identity to canonical seq. 
Fxrefdict = addcolumnconditional(sharedID, tar, 'entry','in1805FalseXref')

In [200]:
Fxrefdict.shape

(3991, 4)

In [201]:

falseidentitytargetTRUE = Fxrefdict[Fxrefdict['in1805FalseXref'] == 'True']
falseidentitytargetTRUE.shape

(1805, 4)

In [202]:
falseidentitytargetFALSE = Fxrefdict[Fxrefdict['in1805FalseXref'] == 'False']
falseidentitytargetFALSE.shape

(2186, 4)

In [203]:
print("number of cys lys to map: ", falseidentitytargetTRUE.labeled_pos_count.sum())
print("number of cys lys lost: ", falseidentitytargetFALSE.labeled_pos_count.sum())
print()
cursum = falseidentitytargetTRUE.labeled_pos_count.sum()
ogsum = tar.labeled_pos_count.sum()
print("origianl count of cys lys before filtering: ", tar.labeled_pos_count.sum())
print("difference btw original amino pos and post filtering = ", ogsum - cursum)

number of cys lys to map:  6830
number of cys lys lost:  8647

origianl count of cys lys before filtering:  15477
difference btw original amino pos and post filtering =  8647


In [206]:
3991 - 1805

2186

In [207]:
falseidentitytargetTRUE

,entry,labeled_pos_count,pos_dict,in1805FalseXref
0,Q09666,112,"{85: 'K', 108: 'C', 128: 'K', 243: 'K', 252: '...",True
1,P21333,75,"{53: 'C', 58: 'K', 87: 'K', 210: 'C', 220: 'K'...",True
2,O75369,65,"{15: 'K', 16: 'K', 26: 'C', 60: 'K', 93: 'K', ...",True
3,Q15149,58,"{191: 'K', 236: 'K', 253: 'K', 317: 'C', 338: ...",True
8,P07900,36,"{58: 'K', 74: 'K', 112: 'K', 185: 'K', 191: 'K...",True
10,P14618,32,"{49: 'C', 62: 'K', 89: 'K', 115: 'K', 125: 'K'...",True
15,P02545,30,"{32: 'K', 78: 'K', 90: 'K', 97: 'K', 108: 'K',...",True
16,O43707,30,"{60: 'C', 98: 'K', 108: 'K', 125: 'K', 173: 'C...",True
19,Q01813,29,"{15: 'K', 25: 'K', 109: 'K', 112: 'C', 123: 'C...",True
20,P11586,29,"{21: 'K', 56: 'K', 66: 'K', 143: 'C', 152: 'C'...",True


In [209]:
COUNTck = falseidentitytargetTRUE.pos_dict.tolist()

In [211]:
# using count() to get count  
# counting C
Cys = 0
Lys = 0
for st in COUNTck:
    C = st.count('C')
    K = st.count('K')
    Cys += C
    Lys += K
print(Cys)
print(Lys)

2955
3875


In [218]:
# using count() to get count  
# counting C
Cys = 0
Lys = 0
for st in truetarls:
    C = st.count('C')
    K = st.count('K')
    Cys += C
    Lys += K
print(Cys)
print(Lys)

6302
9145


In [212]:
truetarget = xrefdict[xrefdict['in3979xref'] == 'True']
print(truetarget.shape)
falsetargets = xrefdict[xrefdict['in3979xref'] == 'False']
print(falsetargets.shape) #12

NameError: name 'xrefdict' is not defined

In [101]:
truetarget.reset_index(drop=True,inplace=True)
falsetargets.reset_index(drop=True,inplace=True)

In [214]:
print("number of cys lys to map: ", truetarget.labeled_pos_count.sum())
print("number of cys lys lost: ", falsetargets.labeled_pos_count.sum())
print()
cursum = truetarget.labeled_pos_count.sum()
ogsum = tar.labeled_pos_count.sum()
print("origianl count of cys lys before filtering: ", tar.labeled_pos_count.sum())
print("difference btw original amino pos and post filtering = ", ogsum - cursum)

number of cys lys to map:  15447


NameError: name 'falsetargets' is not defined

In [217]:
truetarls = truetarget.pos_dict.tolist()

In [215]:
falsetargets.pos_dict.tolist()

NameError: name 'falsetargets' is not defined

In [58]:
# lost 17 K
# lost 13 C
print("origianl perfect remaining: ")
print("number of amino acids : ", 14120/15477)
print("number of uniprot IDs : ",3636/3991 )

origianl perfect remaining: 
number of amino acids :  0.9123215093364347
number of uniprot IDs :  0.9110498621899273


In [59]:
print("FINAL perfect remaining: ")
print("number of amino acids : ", 15447/15477)
print("number of uniprot IDs : ",3979/3991 )

FINAL perfect remaining: 
number of amino acids :  0.9980616398526846
number of uniprot IDs :  0.9969932347782511


# 99% of uniprot ever labeled before IDs and 99% of targetable amino acids (CYS and LYS) available for mis mapping analysis 

 
## value counts of # targets in UniProt proteins:
- 1346 uniprot proteins only have 1 targetable amino acid
- 682 have 2
- 404 have 3 targets
- 265 have 4 targets
- 191 have 5 targets

In [213]:
truetarget.head(5)

,entry,labeled_pos_count,pos_dict,in3979xref
0,Q09666,112,"{85: 'K', 108: 'C', 128: 'K', 243: 'K', 252: '...",True
1,P21333,75,"{53: 'C', 58: 'K', 87: 'K', 210: 'C', 220: 'K'...",True
2,O75369,65,"{15: 'K', 16: 'K', 26: 'C', 60: 'K', 93: 'K', ...",True
3,Q15149,58,"{191: 'K', 236: 'K', 253: 'K', 317: 'C', 338: ...",True
4,P35579,56,"{14: 'K', 29: 'K', 63: 'K', 66: 'K', 79: 'K', ...",True


In [113]:
describeMe(truetarget)

(3979, 4)
Index(['entry', 'labeled_pos_count', 'pos_dict', 'in3979xref'], dtype='object')
entry                0
labeled_pos_count    0
pos_dict             0
in3979xref           0
dtype: int64


In [67]:
numUKBw_numTargets = truetarget.labeled_pos_count.value_counts()
numUKBw_numTargets

1      1486
2       738
3       441
4       287
5       204
6       161
7       124
8       108
9        74
10       65
11       55
12       33
13       30
14       29
15       23
16       19
17       14
20       13
19       13
18       10
21        8
24        7
22        6
29        4
25        4
32        3
23        3
27        2
36        2
30        2
31        2
75        1
26        1
42        1
56        1
58        1
65        1
35        1
43        1
112       1
Name: labeled_pos_count, dtype: int64

In [71]:
# dictionary of cys lys positions
truetarget.to_csv("MISMAP_SCORE_CYS_LYS_15447pos_3979entries.csv", index = False)
# updated number of enteries to number of targets: use for histogram
numUKBw_numTargets.to_csv("MISMAP_SCORE_UniprotCanonical_3979entries_valueCounts_fromDict.csv")

# FILTERING ensembl release fasta2csv files to only include ENSP from each release key

### importing KEYS with ENSP(different # per release) to UKB ID (same 3979) 
- swissprot xref filtered
- labeled uniprot ID 
- uniprot ID shared between all releases
- **ENSP IDs all unique in release keys so 1:many mapping for ENSP to UNIPROT**

```bash
key85.to_csv("key85_10272.csv",index=False)
key92.to_csv("key92_10479.csv",index=False)
key94.to_csv("key94_10699.csv",index=False)
key96.to_csv("key96_10750.csv",index=False)
key97.to_csv("key97_10650.csv",index=False)
```

In [115]:
# importing the KEY for fasta filtering
# turn ENSP column into list
FAS85 = key85['protein_stable_id'].tolist()
FAS92 = key92['protein_stable_id'].tolist()
FAS94 = key94['protein_stable_id'].tolist()
FAS96 = key96['protein_stable_id'].tolist()
FAS97 = key97['protein_stable_id'].tolist() 

In [116]:
# each release fasta2csv file
e85 = pd.read_csv("CYSLYS_failure2map/v85_37_ENSP_fasta.csv")
e92 = pd.read_csv("CYSLYS_failure2map/v92_38_ENSP_fasta.csv")
e94 = pd.read_csv("CYSLYS_failure2map/v94_38_ENSP_fasta.csv")
e96 = pd.read_csv("CYSLYS_failure2map/v96_38_ENSP_fasta.csv")
e97 = pd.read_csv("CYSLYS_failure2map/v97_38_ENSP_fasta.csv")

In [117]:
# computationally $$$

# label fasta ensemb withcol for ENSP matching KEY sets
TF85 = addcolumnconditional(FAS85, e85, 'ENSP','MISMAP_SCORE_ENSP')
TF92 = addcolumnconditional(FAS92, e92, 'ENSP','MISMAP_SCORE_ENSP')
TF94 = addcolumnconditional(FAS94, e94, 'ENSP','MISMAP_SCORE_ENSP')
TF96 = addcolumnconditional(FAS96, e96, 'ENSP','MISMAP_SCORE_ENSP')
TF97 = addcolumnconditional(FAS97, e97, 'ENSP','MISMAP_SCORE_ENSP')

In [118]:
# drop rows that have ENSP not in release KEY
TF85.drop(TF85[TF85["MISMAP_SCORE_ENSP"] == "False"].index, inplace = True)
TF92.drop(TF92[TF92["MISMAP_SCORE_ENSP"] == "False"].index, inplace = True)
TF94.drop(TF94[TF94["MISMAP_SCORE_ENSP"] == "False"].index, inplace = True)
TF96.drop(TF96[TF96["MISMAP_SCORE_ENSP"] == "False"].index, inplace = True)
TF97.drop(TF97[TF97["MISMAP_SCORE_ENSP"] == "False"].index, inplace = True)

In [119]:
dflist = [TF85,TF92,TF94,TF96,TF97]
for i in dflist:
    print(describeMe(i))

(10272, 5)
Index(['ENSPv', 'ENSP', 'Length', 'proSequence', 'MISMAP_SCORE_ENSP'], dtype='object')
ENSPv                0
ENSP                 0
Length               0
proSequence          0
MISMAP_SCORE_ENSP    0
dtype: int64
None
(10479, 5)
Index(['ENSPv', 'ENSP', 'Length', 'proSequence', 'MISMAP_SCORE_ENSP'], dtype='object')
ENSPv                0
ENSP                 0
Length               0
proSequence          0
MISMAP_SCORE_ENSP    0
dtype: int64
None
(10699, 5)
Index(['ENSPv', 'ENSP', 'Length', 'proSequence', 'MISMAP_SCORE_ENSP'], dtype='object')
ENSPv                0
ENSP                 0
Length               0
proSequence          0
MISMAP_SCORE_ENSP    0
dtype: int64
None
(10750, 5)
Index(['ENSPv', 'ENSP', 'Length', 'proSequence', 'MISMAP_SCORE_ENSP'], dtype='object')
ENSPv                0
ENSP                 0
Length               0
proSequence          0
MISMAP_SCORE_ENSP    0
dtype: int64
None
(10650, 5)
Index(['ENSPv', 'ENSP', 'Length', 'proSequence', 'MISMAP_SCORE_EN

In [120]:
TF85.reset_index(inplace=True,drop=True)
TF92.reset_index(inplace=True,drop=True)
TF94.reset_index(inplace=True,drop=True)
TF96.reset_index(inplace=True,drop=True)
TF97.reset_index(inplace=True,drop=True)

# add uniprot  ID column to the ensembl release fasta files using the KEY made from for each release xref file (ENSP - 3979 uniprot IDs)


In [122]:
# going to add uniprot ID using inner merge
# merge on ENSP, add uniprot ID to fasta files
key85.columns = ['ENSP','UniProtSP_xref'] 
key92.columns = ['ENSP','UniProtSP_xref'] 
key94.columns = ['ENSP','UniProtSP_xref'] 
key96.columns = ['ENSP','UniProtSP_xref'] 
key97.columns = ['ENSP','UniProtSP_xref']

In [123]:
def sortbycol(df, colname):
    df.sort_values(by=[colname],inplace=True)
    df.reset_index(inplace=True,drop=True)

In [124]:
dflist = [TF85,TF92,TF94,TF96,TF97, key85, key92, key94, key96, key97]
for i in dflist:
    sortbycol(i, 'ENSP')

In [125]:
fasta85 = pd.merge(TF85,key85,how='inner',on=['ENSP'])
fasta85.head(3)

# mismap_score_ensp just means the ENSP is unique and linked to one of 3979 uniprot IDs

,ENSPv,ENSP,Length,proSequence,MISMAP_SCORE_ENSP,UniProtSP_xref
0,ENSP00000000233.5,ENSP00000000233,180,MGLTVSALFSRIFGKKQMRILMVGLDAAGKTTILYKLKLGEIVTTI...,True,P84085
1,ENSP00000001008.4,ENSP00000001008,459,MTAEEMKATESGAQSAPLPMEGVDISPKQDEGVLKVIKREGTGTEM...,True,Q02790
2,ENSP00000003302.4,ENSP00000003302,1077,MTAELQQDDAAGAADGHGSSCQMLLNQLREITGIQDPSFLHEALKA...,True,Q96RU2


In [126]:
fasta92 = pd.merge(TF92,key92,how='inner',on=['ENSP'])
fasta94 = pd.merge(TF94,key94,how='inner',on=['ENSP'])
fasta96 = pd.merge(TF96,key96,how='inner',on=['ENSP'])
fasta97 = pd.merge(TF97,key97,how='inner',on=['ENSP'])

## so now the ensembl release fasta files with SEQUENCE have an ENSP : UNIPROT ID matching the KEYs made from xref files
- biomart is only for v97
- the xref files were best way to get uniprot mapping from ensembl SOURCE to ensembl sequences

In [127]:
def sortbycol(df, colnamelist):
    df.sort_values(by=colnamelist,inplace=True)
    df.reset_index(inplace=True,drop=True)

In [128]:
# sorting by two columns
colnamelist = ['UniProtSP_xref','ENSP']
sortbycol(fasta85, colnamelist)
sortbycol(fasta92, colnamelist)
sortbycol(fasta94, colnamelist)
sortbycol(fasta96, colnamelist)
sortbycol(fasta97, colnamelist)

In [166]:
fasta85.head(3)

,ENSPv,ENSP,Length,proSequence,MISMAP_SCORE_ENSP,UniProtSP_xref,Identical_UKB_seq
0,ENSP00000313454.4,ENSP00000313454,1052,MEGSEPVAAHQGEEASCSSWGTGSTNKNLPIMSTASVEIDDALYSR...,True,A0AVT1,True
1,ENSP00000399234.2,ENSP00000399234,389,MEGSEPVAAHQGEEASCSSWGTGSTNKNLPIMSTASVEIDDALYSR...,True,A0AVT1,False
2,ENSP00000251527.5,ENSP00000251527,893,MTPPSRAEAGVRRSRVPSEGRWRGAEPPGISASTQPASAGRAARHC...,True,A0FGR8,False


In [129]:
fasta97.equals(fasta85)

False

In [130]:
fasta97.equals(fasta92)

False

In [131]:
fasta97.equals(fasta94)

False

In [132]:
fasta97.equals(fasta96)

False

# saving fasta files containing only ENSP and UNIPROT IDs also contained in KEYS for ENSP---UKB mismapping analysis

- total 15447 cys and lys postions to check in analysis
- used xref release files to make key, so each key is different in shape but identical in # uniprot Unique IDs
- multiple mappign from 1 UKB ID to several ENSP IDs

In [124]:
truetarget.to_csv("MISMAP_SCORE_uniprot_TrueTargets_CYS_LYS_positions_3979.csv", index=False)
truetarget.head(4)

,entry,labeled_pos_count,pos_dict,in3979xref
0,Q09666,112,"{85: 'K', 108: 'C', 128: 'K', 243: 'K', 252: '...",True
1,P21333,75,"{53: 'C', 58: 'K', 87: 'K', 210: 'C', 220: 'K'...",True
2,O75369,65,"{15: 'K', 16: 'K', 26: 'C', 60: 'K', 93: 'K', ...",True
3,Q15149,58,"{191: 'K', 236: 'K', 253: 'K', 317: 'C', 338: ...",True


In [127]:
# renamed and saved as fasta version with added myIdentity score
fasta85.to_csv("MISMAP_SCORE_fasta85_filteredByKEYxref_10272.csv", index=False)  
fasta92.to_csv("MISMAP_SCORE_fasta92_filteredByKEYxref_10479.csv", index=False)  
fasta94.to_csv("MISMAP_SCORE_fasta94_filteredByKEYxref_10699.csv", index=False)  
fasta96.to_csv("MISMAP_SCORE_fasta96_filteredByKEYxref_10750.csv", index=False)  
fasta97.to_csv("MISMAP_SCORE_fasta97_filteredByKEYxref_10650.csv", index=False)

--- 
--- 
--- 
--- ---

## TODO-3: filter UKBccds fasta downloaded 2018 to contain only entries from 3979 unique UKB---ENSP mapping KEYS

```bash
# make dict ID: sequence from my uniprot ccds set
myukb = "CYSLYS_failure2map/countsfromfastaCCDS_filter_normalize_all_AA.csv"

# ensembl fasta files
fasta85
fasta92
fasta94
fasta96
fasta97 
> [ENSPv	ENSP	Length	proSequence	MISMAP_SCORE_ENSP	UniProtSP_xref]

# isoform count file from Uniprot ID mapping: 
- UniprotIDMAPPING_isoformCount__labeledBefore_ENSPsource_3955unique entries.csv

# CYS LYS dictionary positions
truetarget 
> [entry	labeled_pos_count	pos_dict	in3979xref]

```

In [13]:
truetarget = pd.read_csv("MISMAP_SCORE_uniprot_TrueTargets_CYS_LYS_positions_3979.csv")

In [14]:
fasta85 = pd.read_csv("MISMAP_SCORE_fasta85_filteredByKEYxref_10272.csv")  
fasta92 = pd.read_csv("MISMAP_SCORE_fasta92_filteredByKEYxref_10479.csv")  
fasta94 = pd.read_csv("MISMAP_SCORE_fasta94_filteredByKEYxref_10699.csv")  
fasta96 = pd.read_csv("MISMAP_SCORE_fasta96_filteredByKEYxref_10750.csv")  
fasta97 = pd.read_csv("MISMAP_SCORE_fasta97_filteredByKEYxref_10650.csv")

FileNotFoundError: File b'MISMAP_SCORE_fasta85_filteredByKEYxref_10272.csv' does not exist

In [11]:
myukb = pd.read_csv("CYSLYS_failure2map/countsfromfastaCCDS_filter_normalize_all_AA.csv")

In [12]:
print(myukb.columns)
print(myukb.shape)

udf = myukb[['ID', 'Length', 'proSequence','C', 'K']].copy()
print(udf.columns)

Index(['Unnamed: 0', 'ID', 'entryName', 'Length', 'proSequence', 'A', 'B', 'C',
       'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S',
       'T', 'U', 'V', 'W', 'X', 'Y', 'Z'],
      dtype='object')
(18432, 29)
Index(['ID', 'Length', 'proSequence', 'C', 'K'], dtype='object')


In [13]:
# drop all IDs not in 3979 set of labeled and xref release filtering

# grabbing uniprot IDs from 3979 set using truetargets dict file already filtered to contain only these entries
u3979 = truetarget.entry.tolist()
print(u3979[:10])
print(len(u3979))
print(len(set(u3979)))

['Q09666', 'P21333', 'O75369', 'Q15149', 'P35579', 'P13639', 'Q9Y490', 'P49327', 'P07900', 'P00558']
3979
3979


In [14]:
# add col about ID in 3979 set
refUKB = addcolumnconditional(u3979, udf, 'ID','in3979xref')
reference = refUKB[refUKB['in3979xref'] == 'True']
reference.reset_index(drop=True,inplace=True)
print(reference.shape)
print(reference.head(3))

(3979, 6)
       ID  Length                                        proSequence       C  \
0  Q9HAS0     396  MLPSLQESMDGDEKELESSEEGGSAEERRLEPPSSSHYCLYSYRGS...  0.0303   
1  Q86X76     327  MLGFITRPPHRFLSLLCPGLRIPQLSVLCAQPRPRAMAISSSSCEL...  0.0459   
2  Q9NQR4     276  MTSFRLALIQLQISSIKSDNVTRACSFIREAATQGAKIVSLPECFN...  0.0254   

        K in3979xref  
0  0.0581       True  
1  0.0245       True  
2  0.0652       True  


In [15]:
# creating dictionary from ID and Sequence columns
ref_dic = dict(zip(reference.ID, reference.proSequence))

In [16]:
ref_dic

{'Q9HAS0': 'MLPSLQESMDGDEKELESSEEGGSAEERRLEPPSSSHYCLYSYRGSRLAQQRGDSEDGSPSGTNAETPSGDDFSLSLADTNLPSEVEPELRSFIAKRLSRGAVFEGLGNVASVELKIPGYRVGCYYCLFQNEKLLPETVTIDSERNPSEYVVCFLGGSEKGLELFRLELDKYIQGLKNNMNCEARGLESHIKSYLSSWFEDVVCPIQRVVLLFQEKLTFLLHAALSYTPVEVKESDEKTKRDINRFLSVASLQGLIHEGTMTSLCMAMTEEQHKSVVIDCSSSQPQFCNAGSNRFCEDWMQAFLNGAKGGNPFLFRQVLENFKLKAIQDTNNLKRFIRQAEMNHYALFKCYMFLKNCGSGDILLKIVKVEHEEMPEAKNVIAVLEEFMKEALDQSF',
 'Q86X76': 'MLGFITRPPHRFLSLLCPGLRIPQLSVLCAQPRPRAMAISSSSCELPLVAVCQVTSTPDKQQNFKTCAELVREAARLGACLAFLPEAFDFIARDPAETLHLSEPLGGKLLEEYTQLARECGLWLSLGGFHERGQDWEQTQKIYNCHVLLNSKGAVVATYRKTHLCDVEIPGQGPMCESNSTMPGPSLESPVSTPAGKIGLAVCYDMRFPELSLALAQAGAEILTYPSAFGSITGPAHWEVLLRARAIETQCYVVAAAQCGRHHEKRASYGHSMVVDPWGTVVARCSEGPGLCLARIDLNYLRQLRRHLPVFQHRRPDLYGNLGHPLS',
 'Q9NQR4': 'MTSFRLALIQLQISSIKSDNVTRACSFIREAATQGAKIVSLPECFNSPYGAKYFPEYAEKIPGESTQKLSEVAKECSIYLIGGSIPEEDAGKLYNTCAVFGPDGTLLAKYRKIHLFDIDVPGKITFQESKTLSPGDSFSTFDTPYCRVGLGICYDMRFAELAQIYAQRGCQLLVYPGAFNLTTGPAHWELLQRSRAVDNQVYVATASPARDDKASYVAWGHSTVVNPWGEVLAKA

In [18]:
# input df and dictionary with ID and sequence to check
# compares the sequence in df and dict value, adds column based on comparison results
# True is they are identical sequences for same Uniprot ID
# False is they are different, no info on HOW diff they are
def identical2uniprotfasta(dfEnsp, ref_dic):
    newcol = []
    for index, row in dfEnsp.iterrows():
        pep = row['proSequence']
        ukb_id = row['UniProtSP_xref']
        # retrieve uniprot fasta seq using reference dictionary
        mypep = ref_dic[ukb_id]
        str(mypep)
        if mypep == pep:
            newcol.append('True')
        if mypep != pep: 
            newcol.append('False')
    # add identity score list as new column
    dfEnsp.loc[:,'Identical_UKB_seq'] = newcol
    print(dfEnsp.shape)
    return dfEnsp

In [19]:
v85 = identical2uniprotfasta(fasta85, ref_dic)
v92 = identical2uniprotfasta(fasta92, ref_dic)
v94 = identical2uniprotfasta(fasta94, ref_dic)
v96 = identical2uniprotfasta(fasta96, ref_dic)
v97 = identical2uniprotfasta(fasta97, ref_dic)

(10272, 7)
(10479, 7)
(10699, 7)
(10750, 7)
(10650, 7)


In [20]:
checkColumnValues(v85, "Identical_UKB_seq")
print() 
checkColumnValues(v92, "Identical_UKB_seq")
print() 
checkColumnValues(v94, "Identical_UKB_seq")
print() 
checkColumnValues(v96, "Identical_UKB_seq")
print() 
checkColumnValues(v97, "Identical_UKB_seq")

  Identical_UKB_seq  Count
0              True   5925
1             False   4347

  Identical_UKB_seq  Count
0              True   5957
1             False   4522

  Identical_UKB_seq  Count
0              True   6097
1             False   4602

  Identical_UKB_seq  Count
0              True   6127
1             False   4623

  Identical_UKB_seq  Count
0              True   6052
1             False   4598


In [22]:
v85.to_csv("MISMAP_SCORE_fasta85_CanonicalUKB_identitycol_10272.csv")  
v92.to_csv("MISMAP_SCORE_fasta92_CanonicalUKB_identitycol_10479.csv")  
v94.to_csv("MISMAP_SCORE_fasta94_CanonicalUKB_identitycol_10699.csv")  
v96.to_csv("MISMAP_SCORE_fasta96_CanonicalUKB_identitycol_10750.csv")  
v97.to_csv("MISMAP_SCORE_fasta97_CanonicalUKB_identitycol_10650.csv")

---
---
---
---


---
---
---
---


---
---
---
---


---
---
---
---


In [23]:
v97.head(3)

,ENSPv,ENSP,Length,proSequence,MISMAP_SCORE_ENSP,UniProtSP_xref,Identical_UKB_seq
0,ENSP00000313454.4,ENSP00000313454,1052,MEGSEPVAAHQGEEASCSSWGTGSTNKNLPIMSTASVEIDDALYSR...,True,A0AVT1,True
1,ENSP00000399234.2,ENSP00000399234,389,MEGSEPVAAHQGEEASCSSWGTGSTNKNLPIMSTASVEIDDALYSR...,True,A0AVT1,False
2,ENSP00000251527.6,ENSP00000251527,845,MSGARGEGPEAGAGGAGGRAAPENPGGVLSVELPGLLAQLARSFAL...,True,A0FGR8,False


# saving release fasta files in 2 forms: 
- ENSP with sequences identical to uniprot canonical fasta
- ENSP with sequences not matching uniprot canonical fasta

- ENSP IDs with identical seq to uniprot made into list and used to compare my identity score to xref file provided score (see if v85 is really a seq difference result)


In [24]:
# version of fasta release files with only sequences that match canonical uniprot 
v85TRUE = v85[v85['Identical_UKB_seq'] == 'True']
v85TRUE.reset_index(drop=True,inplace=True)
print(v85TRUE.shape)

v92TRUE = v92[v92['Identical_UKB_seq'] == 'True']
v92TRUE.reset_index(drop=True,inplace=True)
print(v92TRUE.shape)

v94TRUE = v94[v94['Identical_UKB_seq'] == 'True']
v94TRUE.reset_index(drop=True,inplace=True)
print(v94TRUE.shape)

v96TRUE = v96[v96['Identical_UKB_seq'] == 'True']
v96TRUE.reset_index(drop=True,inplace=True)
print(v96TRUE.shape)

v97TRUE = v97[v97['Identical_UKB_seq'] == 'True']
v97TRUE.reset_index(drop=True,inplace=True)
print(v97TRUE.shape)
"""
files for TRUE match canonical seq. saved as...
v85_Ensembl_fasta_identical_canonicalUKB_5925.csv
v92_Ensembl_fasta_identical_canonicalUKB_5957.csv
v94_Ensembl_fasta_identical_canonicalUKB_6097.csv
v96_Ensembl_fasta_identical_canonicalUKB_6127.csv
v97_Ensembl_fasta_identical_canonicalUKB_6052.csv
"""


(5925, 7)
(5957, 7)
(6097, 7)
(6127, 7)
(6052, 7)


'\nfiles for TRUE match canonical seq. saved as...\nv85_Ensembl_fasta_identical_canonicalUKB_5925.csv\nv92_Ensembl_fasta_identical_canonicalUKB_5957.csv\nv94_Ensembl_fasta_identical_canonicalUKB_6097.csv\nv96_Ensembl_fasta_identical_canonicalUKB_6127.csv\nv97_Ensembl_fasta_identical_canonicalUKB_6052.csv\n'

In [25]:
# version of fasta release files with only sequences that do not match canonical uniprot
v85FALSE = v85[v85['Identical_UKB_seq'] == 'False']
v85FALSE.reset_index(drop=True,inplace=True)
print(v85FALSE.shape)

v92FALSE = v92[v92['Identical_UKB_seq'] == 'False']
v92FALSE.reset_index(drop=True,inplace=True)
print(v92FALSE.shape)

v94FALSE = v94[v94['Identical_UKB_seq'] == 'False']
v94FALSE.reset_index(drop=True,inplace=True)
print(v94FALSE.shape)

v96FALSE = v96[v96['Identical_UKB_seq'] == 'False']
v96FALSE.reset_index(drop=True,inplace=True)
print(v96FALSE.shape)

v97FALSE = v97[v97['Identical_UKB_seq'] == 'False']
v97FALSE.reset_index(drop=True,inplace=True)
print(v97FALSE.shape)

(4347, 7)
(4522, 7)
(4602, 7)
(4623, 7)
(4598, 7)


In [27]:
v85FALSE.to_csv("v85_Ensembl_fasta_NOTidentical_canonicalUKB_4347.csv",index=False)
v92FALSE.to_csv("v92_Ensembl_fasta_NOTidentical_canonicalUKB_4522.csv",index=False)
v94FALSE.to_csv("v94_Ensembl_fasta_NOTidentical_canonicalUKB_4602.csv",index=False)
v96FALSE.to_csv("v96_Ensembl_fasta_NOTidentical_canonicalUKB_4623.csv",index=False)
v97FALSE.to_csv("v97_Ensembl_fasta_NOTidentical_canonicalUKB_4598.csv",index=False)

## Comparison of xref identity scores to my own scores made by comparing release fasta files (all have same 3979 uniprot IDs with diff # of unique ENSP multimapping) showed for the GRCh38 files the 2 identity scores are about the same
- v85 was highest identity from my alignment with 57% seq. matching canonical uniprot 
- other releases have about 56% of sequences matching

---
---
---
---


---
---
---
---


In [70]:
# importing updated csv of releases

# ensembl TRUE AND FALSE all rows with distance scores
v85 = pd.read_csv("MISMAP_SCORE_fasta85_CanonicalUKB_identitycol_distanceScores_10272.csv")  
v92 = pd.read_csv("MISMAP_SCORE_fasta92_CanonicalUKB_identitycol_distanceScores_10479.csv")  
v94 = pd.read_csv("MISMAP_SCORE_fasta94_CanonicalUKB_identitycol_distanceScores_10699.csv")  
v96 = pd.read_csv("MISMAP_SCORE_fasta96_CanonicalUKB_identitycol_distanceScores_10750.csv")  
v97 = pd.read_csv("MISMAP_SCORE_fasta97_CanonicalUKB_identitycol_distanceScores_10650.csv")

In [71]:
v85.drop(['Unnamed: 0'],inplace=True,axis=1)
v92.drop(['Unnamed: 0'],inplace=True,axis=1)
v94.drop(['Unnamed: 0'],inplace=True,axis=1)
v96.drop(['Unnamed: 0'],inplace=True,axis=1)
v97.drop(['Unnamed: 0'],inplace=True,axis=1)

# How safe is stable ID mapping- within ensembl fasta release files only - stable ID over time data 



```python 
# variables to use
v85FALSE
v92FALSE
v94FALSE
v96FALSE
v97FALSE
v85TRUE
v92TRUE
v94TRUE
v96TRUE
v97TRUE
v85
v92
v94
v96
v97
```

In [72]:
v85.head(5)

,ENSPv,ENSP,Length,proSequence,MISMAP_SCORE_ENSP,UniProtSP_xref,Identical_UKB_seq,matchedUKBcanonical,hammingScore,hamNormalizedScore,levenshteinScore
0,ENSP00000313454.4,ENSP00000313454,1052,MEGSEPVAAHQGEEASCSSWGTGSTNKNLPIMSTASVEIDDALYSR...,True,A0AVT1,True,True,0,0.000000,0
1,ENSP00000399234.2,ENSP00000399234,389,MEGSEPVAAHQGEEASCSSWGTGSTNKNLPIMSTASVEIDDALYSR...,True,A0AVT1,False,False,684,0.650190,663
2,ENSP00000251527.5,ENSP00000251527,893,MTPPSRAEAGVRRSRVPSEGRWRGAEPPGISASTQPASAGRAARHC...,True,A0FGR8,False,False,859,0.932682,70
3,ENSP00000279907.7,ENSP00000279907,1464,MAGIIKKQILKHLSRFTKNLSPDKINLSTLKGEGELKNLELDEEVL...,True,A0JNW5,True,True,0,0.000000,0
4,ENSP00000349285.3,ENSP00000349285,522,MAGIIKKQILKHLSRFTKNLSPDKINLSTLKGEGELKNLELDEEVL...,True,A0JNW5,False,False,948,0.647541,942


In [18]:
v97.head(5)

,ENSPv,ENSP,Length,proSequence,MISMAP_SCORE_ENSP,UniProtSP_xref,Identical_UKB_seq,matchedUKBcanonical,hammingScore,hamNormalizedScore,levenshteinScore
0,ENSP00000313454.4,ENSP00000313454,1052,MEGSEPVAAHQGEEASCSSWGTGSTNKNLPIMSTASVEIDDALYSR...,True,A0AVT1,True,True,0,0.000000,0
1,ENSP00000399234.2,ENSP00000399234,389,MEGSEPVAAHQGEEASCSSWGTGSTNKNLPIMSTASVEIDDALYSR...,True,A0AVT1,False,False,684,0.650190,663
2,ENSP00000251527.6,ENSP00000251527,845,MSGARGEGPEAGAGGAGGRAAPENPGGVLSVELPGLLAQLARSFAL...,True,A0FGR8,False,False,859,0.932682,77
3,ENSP00000279907.7,ENSP00000279907,1464,MAGIIKKQILKHLSRFTKNLSPDKINLSTLKGEGELKNLELDEEVL...,True,A0JNW5,True,True,0,0.000000,0
4,ENSP00000349285.3,ENSP00000349285,522,MAGIIKKQILKHLSRFTKNLSPDKINLSTLKGEGELKNLELDEEVL...,True,A0JNW5,False,False,948,0.647541,942


In [19]:
# make all release files into list for ENSP ID then make set to find what set overlaps all releases
ensp85 = set(v85['ENSP'].tolist())
ensp92 = set(v92['ENSP'].tolist())
ensp94 = set(v94['ENSP'].tolist())
ensp96 = set(v96['ENSP'].tolist())
ensp97 = set(v97['ENSP'].tolist())

In [20]:
sharedENSP = ensp85 & ensp92 & ensp94 & ensp96 & ensp97
print(len(sharedENSP))
print(len(set(sharedENSP)))
sharedENSP = list(sharedENSP)

8939
8939


In [22]:
# label each fasta release file with col for ENSPacrossAllRel
def addcolumnconditional(mapList, df, dfcol, newcol): 
    mendel = []
    for g in df[dfcol]:
        if g in mapList: 
            mendel.append("True")
        else:
            mendel.append("False")
    df.loc[:,newcol] = mendel
    df.drop(df[df[newcol] == "False"].index, inplace = True)
    df.reset_index(inplace=True, drop=True)
    return df

In [23]:
enspv85 = addcolumnconditional(sharedENSP, v85, 'ENSP','ENSPinAllReleases')
enspv92 = addcolumnconditional(sharedENSP, v92, 'ENSP','ENSPinAllReleases')
enspv94 = addcolumnconditional(sharedENSP, v94, 'ENSP','ENSPinAllReleases')
enspv96 = addcolumnconditional(sharedENSP, v96, 'ENSP','ENSPinAllReleases')
enspv97 = addcolumnconditional(sharedENSP, v97, 'ENSP','ENSPinAllReleases')

In [74]:
print(v85.shape)
print(v92.shape)
print(v94.shape)
print(v96.shape)
print(v97.shape)

(10272, 11)
(10479, 11)
(10699, 11)
(10750, 11)
(10650, 11)


In [73]:
print(enspv85.shape)
print(enspv92.shape)
print(enspv94.shape)
print(enspv96.shape)
print(enspv97.shape)

(8939, 16)
(8939, 16)
(8939, 16)
(8939, 16)
(8939, 16)


In [25]:
enspv96.head(10)

,ENSPv,ENSP,Length,proSequence,MISMAP_SCORE_ENSP,UniProtSP_xref,Identical_UKB_seq,matchedUKBcanonical,hammingScore,hamNormalizedScore,levenshteinScore,ENSPinAllReleases
0,ENSP00000313454.4,ENSP00000313454,1052,MEGSEPVAAHQGEEASCSSWGTGSTNKNLPIMSTASVEIDDALYSR...,True,A0AVT1,True,True,0,0.000000,0,True
1,ENSP00000399234.2,ENSP00000399234,389,MEGSEPVAAHQGEEASCSSWGTGSTNKNLPIMSTASVEIDDALYSR...,True,A0AVT1,False,False,684,0.650190,663,True
2,ENSP00000251527.6,ENSP00000251527,845,MSGARGEGPEAGAGGAGGRAAPENPGGVLSVELPGLLAQLARSFAL...,True,A0FGR8,False,False,859,0.932682,77,True
3,ENSP00000279907.7,ENSP00000279907,1464,MAGIIKKQILKHLSRFTKNLSPDKINLSTLKGEGELKNLELDEEVL...,True,A0JNW5,True,True,0,0.000000,0,True
4,ENSP00000349285.3,ENSP00000349285,522,MAGIIKKQILKHLSRFTKNLSPDKINLSTLKGEGELKNLELDEEVL...,True,A0JNW5,False,False,948,0.647541,942,True
5,ENSP00000260777.11,ENSP00000260777,571,MVIEEVNFMQNHLEIEKTCRESAEALATKLNKENKTLKRISMLYMA...,True,A0MZ66,False,False,583,0.923930,60,True
6,ENSP00000347532.4,ENSP00000347532,631,MNSSDEEKQLQLITSLKEQAIGEYEDLRAENQKTKEKCDKIRQERD...,True,A0MZ66,True,True,0,0.000000,0,True
7,ENSP00000376635.4,ENSP00000376635,498,MVIEEVNFMQNHLEIEKTCRESAEALATKLNKENKTLKRISMLYMA...,True,A0MZ66,False,False,587,0.930269,133,True
8,ENSP00000376636.3,ENSP00000376636,558,MNSSDEEKQLQLITSLKEQAIGEYEDLRAENQKTKEKCDKIRQERD...,True,A0MZ66,False,False,73,0.115689,73,True
9,ENSP00000374565.3,ENSP00000374565,195,MNPVYSPGSSGVPYANAKGIGYPAGFPMGYAAAAPAYSPNMYPGAN...,True,A1KXE4,True,True,0,0.000000,0,True


In [26]:
enspv85.columns

Index(['ENSPv', 'ENSP', 'Length', 'proSequence', 'MISMAP_SCORE_ENSP',
       'UniProtSP_xref', 'Identical_UKB_seq', 'matchedUKBcanonical',
       'hammingScore', 'hamNormalizedScore', 'levenshteinScore',
       'ENSPinAllReleases'],
      dtype='object')

In [27]:
dflist = [enspv85, enspv92, enspv94, enspv96, enspv97]
for i in dflist:
    sortbycol(i, 'ENSP')

## how many uniprot IDs represented in shared 8939 IDs == 3914 uniprot IDs
- 65 IDs lost when controlling for same ENSP IDS and same UKB IDs across all release files
- with 3979, only controlled for same Uniprot IDs, files have different # unique ENSP multi mapping to same Uniprot IDs

In [28]:
3979 - 3914

65

### quality check: all uniprot IDs mapped to same 8938 ENSP shared in releases should equal...and they do

In [29]:
# make each release file that contains only shared ENSP IDs
ukb8939_85 = set(enspv85['UniProtSP_xref'].tolist())
ukb8939_92 = set(enspv92['UniProtSP_xref'].tolist())
ukb8939_94 = set(enspv94['UniProtSP_xref'].tolist())
ukb8939_96 = set(enspv96['UniProtSP_xref'].tolist())
ukb8939_97 = set(enspv97['UniProtSP_xref'].tolist())

In [30]:
ukb8939_85 == ukb8939_92

True

In [31]:
ukb8939_85 == ukb8939_94

True

In [32]:
ukb8939_85 == ukb8939_96

True

In [33]:
ukb8939_85 == ukb8939_97

True

In [34]:
print(len(ukb8939_85))

3914


## comparing  ENSP IDs shared between all releases, do they all have the same prosequence? does 85 have the most difference compared to other GRCh38 sequences? 
- 85 has most difference
- all ensp IDs have at least 1 different protein sequence with another release

In [37]:
enspv92.columns

Index(['ENSPv', 'ENSP', 'Length', 'proSequence', 'MISMAP_SCORE_ENSP',
       'UniProtSP_xref', 'Identical_UKB_seq', 'matchedUKBcanonical',
       'hammingScore', 'hamNormalizedScore', 'levenshteinScore',
       'ENSPinAllReleases'],
      dtype='object')

In [38]:
# first way of testing differences
test85_92 = enspv85.where(enspv85.proSequence==enspv92.proSequence).notna()
checkColumnValues(test85_92,'proSequence')

   proSequence  Count
0         True   8807
1        False    132


In [39]:
test85_94 = enspv85.where(enspv85.proSequence==enspv94.proSequence).notna()
checkColumnValues(test85_94,'proSequence')

   proSequence  Count
0         True   8806
1        False    133


In [40]:
test85_96 = enspv85.where(enspv85.proSequence==enspv96.proSequence).notna()
checkColumnValues(test85_96,'proSequence')

   proSequence  Count
0         True   8787
1        False    152


In [41]:
test85_97 = enspv85.where(enspv85.proSequence==enspv97.proSequence).notna()
checkColumnValues(test85_97,'proSequence')

   proSequence  Count
0         True   8776
1        False    163


In [42]:
# more robust way of testing differences
# creating dictionary from ENSP and Sequence columns in all release files
# all have the same ENSP IDs - 8939
dict_85 = dict(zip(enspv85.ENSP, enspv85.proSequence))
dict_92 = dict(zip(enspv92.ENSP, enspv92.proSequence))
dict_94 = dict(zip(enspv94.ENSP, enspv94.proSequence))
dict_96 = dict(zip(enspv96.ENSP, enspv96.proSequence))
dict_97 = dict(zip(enspv97.ENSP, enspv97.proSequence))

In [47]:
def Average(ls): 
    avg =  mean(ls) 
    finalavg = round(avg,2)
    return finalavg
def identicalSequences_inEnsembl(dfEnsp, ref_dic, newcolresult):
    res = []
    ham = []
    hamN = []
    lev = []
    serSeq = dfEnsp['proSequence'].copy()
    serID = dfEnsp['ENSP'].copy()
    for inx, val in serSeq.items():
        pep = str(val)
        p = pep.strip()
        idd = str(serID[inx])
        # check pep to dict pep sequence
        mypep = ref_dic[idd]
        str(mypep)
        #mypep = mypep.strip()
        # identical
        if mypep == p:
            res.append('True')
        # not identical to other release ENSP
        # only does score for different sequences
        if mypep != p: 
            res.append('False')
            # calc. distance scores for enesembl differing sequences
            ham.append(textdistance.hamming(mypep, p))
            # normalized hamming = # mismatched positions/ len of longer sequence
            hamN.append(textdistance.hamming.normalized_distance(mypep, p))
            # levenshtein score is edit based but not not penalized position, insertion at pos 1 is jsut 1 diff
            lev.append(textdistance.levenshtein(mypep,p))
    
    # add result col of seq match
    dfEnsp.loc[:,newcolresult] = res
    # select only False to matching seq rows
    dfFalse = dfEnsp[dfEnsp[newcolresult] == 'False']
    dfFALSE_list = dfFalse['ENSP'].tolist()
    print("ENSP IDs that have different protein sequences btw releases: ", dfFALSE_list)
    checkColumnValues(dfEnsp, newcolresult)

    print("hamming score average of different ensp sequences: ", Average(ham))
    print("hamming Normalized score average of different ensp sequences: ", Average(hamN))
    print("levenshtein score average of different ensp sequences: ", Average(lev))

    # this code was for uniprot seq comparisons
    # hamaverage = np.mean(dfFalse.hammingScore)
    # print("hamming Score average : ", hamaverage)
    # hamnormAverage = np.mean(dfFalse.hamNormalizedScore)
    # print("hamming normalized average : ", hamnormAverage)
    # levaverage = np.mean(dfFalse.levenshteinScore)
    # print("levenshtein Score average : ", levaverage)
    print()
    print("returns False for pro seq match dataframe")

    return dfFalse

In [48]:
result_92vs85 = identicalSequences_inEnsembl(enspv92, dict_85, 'proSeq_sameas85')
result_94vs85 = identicalSequences_inEnsembl(enspv94, dict_85, 'proSeq_sameas85')
result_96vs85 = identicalSequences_inEnsembl(enspv96, dict_85, 'proSeq_sameas85')
result_97vs85 = identicalSequences_inEnsembl(enspv97, dict_85, 'proSeq_sameas85')

ENSP IDs that have different protein sequences btw releases:  ['ENSP00000156825', 'ENSP00000216780', 'ENSP00000222224', 'ENSP00000225573', 'ENSP00000233242', 'ENSP00000250377', 'ENSP00000251157', 'ENSP00000260777', 'ENSP00000262878', 'ENSP00000262887', 'ENSP00000263036', 'ENSP00000264108', 'ENSP00000264202', 'ENSP00000264904', 'ENSP00000265447', 'ENSP00000266027', 'ENSP00000278616', 'ENSP00000279146', 'ENSP00000289473', 'ENSP00000291281', 'ENSP00000300648', 'ENSP00000313098', 'ENSP00000313420', 'ENSP00000319361', 'ENSP00000322938', 'ENSP00000324697', 'ENSP00000325677', 'ENSP00000328690', 'ENSP00000331023', 'ENSP00000336775', 'ENSP00000337598', 'ENSP00000338552', 'ENSP00000339247', 'ENSP00000341292', 'ENSP00000341730', 'ENSP00000342035', 'ENSP00000342351', 'ENSP00000345881', 'ENSP00000349486', 'ENSP00000350199', 'ENSP00000350563', 'ENSP00000350846', 'ENSP00000351159', 'ENSP00000351483', 'ENSP00000352575', 'ENSP00000353483', 'ENSP00000354907', 'ENSP00000357679', 'ENSP00000358832', 'ENSP0

ENSP IDs that have different protein sequences btw releases:  ['ENSP00000156825', 'ENSP00000216780', 'ENSP00000222224', 'ENSP00000225573', 'ENSP00000228284', 'ENSP00000229854', 'ENSP00000233242', 'ENSP00000250377', 'ENSP00000251157', 'ENSP00000251527', 'ENSP00000256578', 'ENSP00000260777', 'ENSP00000262878', 'ENSP00000262887', 'ENSP00000263036', 'ENSP00000263741', 'ENSP00000264108', 'ENSP00000264202', 'ENSP00000264904', 'ENSP00000265447', 'ENSP00000265537', 'ENSP00000266027', 'ENSP00000278616', 'ENSP00000279146', 'ENSP00000286091', 'ENSP00000289473', 'ENSP00000291281', 'ENSP00000300648', 'ENSP00000300933', 'ENSP00000313098', 'ENSP00000313420', 'ENSP00000319361', 'ENSP00000322938', 'ENSP00000324697', 'ENSP00000325677', 'ENSP00000328690', 'ENSP00000331023', 'ENSP00000336775', 'ENSP00000337598', 'ENSP00000338552', 'ENSP00000338806', 'ENSP00000339247', 'ENSP00000340824', 'ENSP00000341292', 'ENSP00000341730', 'ENSP00000342035', 'ENSP00000342351', 'ENSP00000345230', 'ENSP00000345881', 'ENSP0

In [49]:
result_85vs92 = identicalSequences_inEnsembl(enspv85, dict_92, 'proSeq_sameas92')
result_94vs92 = identicalSequences_inEnsembl(enspv94, dict_92, 'proSeq_sameas92')
result_96vs92 = identicalSequences_inEnsembl(enspv96, dict_92, 'proSeq_sameas92')
result_97vs92 = identicalSequences_inEnsembl(enspv97, dict_92, 'proSeq_sameas92')

ENSP IDs that have different protein sequences btw releases:  ['ENSP00000156825', 'ENSP00000216780', 'ENSP00000222224', 'ENSP00000225573', 'ENSP00000233242', 'ENSP00000250377', 'ENSP00000251157', 'ENSP00000260777', 'ENSP00000262878', 'ENSP00000262887', 'ENSP00000263036', 'ENSP00000264108', 'ENSP00000264202', 'ENSP00000264904', 'ENSP00000265447', 'ENSP00000266027', 'ENSP00000278616', 'ENSP00000279146', 'ENSP00000289473', 'ENSP00000291281', 'ENSP00000300648', 'ENSP00000313098', 'ENSP00000313420', 'ENSP00000319361', 'ENSP00000322938', 'ENSP00000324697', 'ENSP00000325677', 'ENSP00000328690', 'ENSP00000331023', 'ENSP00000336775', 'ENSP00000337598', 'ENSP00000338552', 'ENSP00000339247', 'ENSP00000341292', 'ENSP00000341730', 'ENSP00000342035', 'ENSP00000342351', 'ENSP00000345881', 'ENSP00000349486', 'ENSP00000350199', 'ENSP00000350563', 'ENSP00000350846', 'ENSP00000351159', 'ENSP00000351483', 'ENSP00000352575', 'ENSP00000353483', 'ENSP00000354907', 'ENSP00000357679', 'ENSP00000358832', 'ENSP0

In [50]:
result_85vs94 = identicalSequences_inEnsembl(enspv85, dict_94, 'proSeq_sameas94')
result_92vs94 = identicalSequences_inEnsembl(enspv92, dict_94, 'proSeq_sameas94')
result_96vs94 = identicalSequences_inEnsembl(enspv96, dict_94, 'proSeq_sameas94')
result_97vs94 = identicalSequences_inEnsembl(enspv97, dict_94, 'proSeq_sameas94')

ENSP IDs that have different protein sequences btw releases:  ['ENSP00000156825', 'ENSP00000216780', 'ENSP00000222224', 'ENSP00000225573', 'ENSP00000233242', 'ENSP00000250377', 'ENSP00000251157', 'ENSP00000260777', 'ENSP00000262878', 'ENSP00000262887', 'ENSP00000263036', 'ENSP00000264108', 'ENSP00000264202', 'ENSP00000264904', 'ENSP00000265447', 'ENSP00000266027', 'ENSP00000278616', 'ENSP00000279146', 'ENSP00000289473', 'ENSP00000291281', 'ENSP00000300648', 'ENSP00000313098', 'ENSP00000313420', 'ENSP00000319361', 'ENSP00000322938', 'ENSP00000324697', 'ENSP00000325677', 'ENSP00000328690', 'ENSP00000331023', 'ENSP00000336775', 'ENSP00000337598', 'ENSP00000338552', 'ENSP00000339247', 'ENSP00000341292', 'ENSP00000341730', 'ENSP00000342035', 'ENSP00000342351', 'ENSP00000345881', 'ENSP00000349486', 'ENSP00000350199', 'ENSP00000350563', 'ENSP00000350846', 'ENSP00000351159', 'ENSP00000351483', 'ENSP00000352575', 'ENSP00000353483', 'ENSP00000354907', 'ENSP00000357679', 'ENSP00000358832', 'ENSP0

In [51]:
result_85vs96 = identicalSequences_inEnsembl(enspv85, dict_96, 'proSeq_sameas96')
result_92vs96 = identicalSequences_inEnsembl(enspv92, dict_96, 'proSeq_sameas96')
result_94vs96 = identicalSequences_inEnsembl(enspv94, dict_96, 'proSeq_sameas96')
result_97vs96 = identicalSequences_inEnsembl(enspv97, dict_96, 'proSeq_sameas96')

ENSP IDs that have different protein sequences btw releases:  ['ENSP00000156825', 'ENSP00000216780', 'ENSP00000222224', 'ENSP00000225573', 'ENSP00000228284', 'ENSP00000229854', 'ENSP00000233242', 'ENSP00000250377', 'ENSP00000251157', 'ENSP00000251527', 'ENSP00000260777', 'ENSP00000262878', 'ENSP00000262887', 'ENSP00000263036', 'ENSP00000264108', 'ENSP00000264202', 'ENSP00000264904', 'ENSP00000265447', 'ENSP00000265537', 'ENSP00000266027', 'ENSP00000278616', 'ENSP00000279146', 'ENSP00000286091', 'ENSP00000289473', 'ENSP00000291281', 'ENSP00000300648', 'ENSP00000313098', 'ENSP00000313420', 'ENSP00000319361', 'ENSP00000322938', 'ENSP00000324697', 'ENSP00000325677', 'ENSP00000328690', 'ENSP00000331023', 'ENSP00000336775', 'ENSP00000337598', 'ENSP00000338552', 'ENSP00000338806', 'ENSP00000339247', 'ENSP00000341292', 'ENSP00000341730', 'ENSP00000342035', 'ENSP00000342351', 'ENSP00000345881', 'ENSP00000347847', 'ENSP00000349486', 'ENSP00000350199', 'ENSP00000350563', 'ENSP00000350846', 'ENSP0

In [52]:
result_85vs97 = identicalSequences_inEnsembl(enspv85, dict_97, 'proSeq_sameas97')
result_92vs97 = identicalSequences_inEnsembl(enspv92, dict_97, 'proSeq_sameas97')
result_94vs97 = identicalSequences_inEnsembl(enspv94, dict_97, 'proSeq_sameas97')
result_96vs97 = identicalSequences_inEnsembl(enspv96, dict_97, 'proSeq_sameas97')

ENSP IDs that have different protein sequences btw releases:  ['ENSP00000156825', 'ENSP00000216780', 'ENSP00000222224', 'ENSP00000225573', 'ENSP00000228284', 'ENSP00000229854', 'ENSP00000233242', 'ENSP00000250377', 'ENSP00000251157', 'ENSP00000251527', 'ENSP00000256578', 'ENSP00000260777', 'ENSP00000262878', 'ENSP00000262887', 'ENSP00000263036', 'ENSP00000263741', 'ENSP00000264108', 'ENSP00000264202', 'ENSP00000264904', 'ENSP00000265447', 'ENSP00000265537', 'ENSP00000266027', 'ENSP00000278616', 'ENSP00000279146', 'ENSP00000286091', 'ENSP00000289473', 'ENSP00000291281', 'ENSP00000300648', 'ENSP00000300933', 'ENSP00000313098', 'ENSP00000313420', 'ENSP00000319361', 'ENSP00000322938', 'ENSP00000324697', 'ENSP00000325677', 'ENSP00000328690', 'ENSP00000331023', 'ENSP00000336775', 'ENSP00000337598', 'ENSP00000338552', 'ENSP00000338806', 'ENSP00000339247', 'ENSP00000340824', 'ENSP00000341292', 'ENSP00000341730', 'ENSP00000342035', 'ENSP00000342351', 'ENSP00000345230', 'ENSP00000345881', 'ENSP0

# all output saved to AllENSPcommon8939_proSequence_differences.txt
    - dir_M/TSV../AllENSPcommon8939_proSequence_differences.txt

# are uniprot canonical sequences represetned by at least 1 ensp in each release?

## save the ENSP IDs that do not have a canonical match in each release, save ENSP IDs that do not have canonical match in ALL releases

In [54]:
dflist = [enspv85, enspv92, enspv94, enspv96, enspv97]
for i in dflist:
    sortbycol(i, 'ENSP')

In [75]:
v85ALL_ukbset = set(v85['UniProtSP_xref'].tolist())
v92ALL_ukbset = set(v92['UniProtSP_xref'].tolist())
v94ALL_ukbset = set(v94['UniProtSP_xref'].tolist())
v96ALL_ukbset = set(v96['UniProtSP_xref'].tolist())
v97ALL_ukbset = set(v97['UniProtSP_xref'].tolist())

In [76]:
print(len(v85ALL_ukbset))
print(len(v92ALL_ukbset))
print(len(v94ALL_ukbset))
print(len(v96ALL_ukbset))
print(len(v97ALL_ukbset))

3979
3979
3979
3979
3979


In [81]:
v97ALL_ukbset == v85ALL_ukbset

True

In [82]:
all3979 = list(v97ALL_ukbset)

In [56]:
v85TRUE = pd.read_csv("v85_editDistance_ensemblFasta_canonicalUKBsequence_identityScore_5925.csv")
v92TRUE = pd.read_csv("v92_editDistance_ensemblFasta_canonicalUKBsequence_identityScore_5957.csv")
v94TRUE = pd.read_csv("v94_editDistance_ensemblFasta_canonicalUKBsequence_identityScore_6097.csv")
v96TRUE = pd.read_csv("v96_editDistance_ensemblFasta_canonicalUKBsequence_identityScore_6127.csv")
v97TRUE = pd.read_csv("v97_editDistance_ensemblFasta_canonicalUKBsequence_identityScore_6052.csv")

In [58]:
v85TRUE.columns

Index(['ENSPv', 'ENSP', 'Length', 'proSequence', 'MISMAP_SCORE_ENSP',
       'UniProtSP_xref', 'Identical_UKB_seq', 'matchedUKBcanonical',
       'hammingScore', 'hamNormalizedScore', 'levenshteinScore'],
      dtype='object')

In [77]:
v85TRUE_ukbset = set(v85TRUE['UniProtSP_xref'].tolist())
v92TRUE_ukbset = set(v92TRUE['UniProtSP_xref'].tolist())
v94TRUE_ukbset = set(v94TRUE['UniProtSP_xref'].tolist())
v96TRUE_ukbset = set(v96TRUE['UniProtSP_xref'].tolist())
v97TRUE_ukbset = set(v97TRUE['UniProtSP_xref'].tolist())

In [78]:
print(len(v85TRUE_ukbset))
print(len(v92TRUE_ukbset))
print(len(v94TRUE_ukbset))
print(len(v96TRUE_ukbset))
print(len(v97TRUE_ukbset))
v85TRUE_ukbset = list(v85TRUE_ukbset)
v92TRUE_ukbset = list(v92TRUE_ukbset)
v94TRUE_ukbset = list(v94TRUE_ukbset)
v96TRUE_ukbset = list(v96TRUE_ukbset)
v97TRUE_ukbset = list(v97TRUE_ukbset)

3884
3878
3895
3890
3885


In [79]:
print(len(v85TRUE_ukbset) - 3979)
print(len(v92TRUE_ukbset) - 3979)
print(len(v94TRUE_ukbset) - 3979)
print(len(v96TRUE_ukbset) - 3979)
print(len(v97TRUE_ukbset) - 3979)

-95
-101
-84
-89
-94


In [ ]:
# which uniprot IDs do not have canonical seq match from each release
def diffList(L1, L2):
    return list(set(L1) - set(L2))


In [85]:
print(len(diffList(all3979,v85TRUE_ukbset)))

95


In [86]:
print(len(diffList(all3979,v92TRUE_ukbset)))

101


In [87]:
print(len(diffList(all3979,v94TRUE_ukbset)))

84


In [88]:
print(len(diffList(all3979,v96TRUE_ukbset)))

89


In [89]:
print(len(diffList(all3979,v97TRUE_ukbset)))

94


In [96]:
v85_nomatch = diffList(all3979,v85TRUE_ukbset)

In [97]:
v92_nomatch = diffList(all3979,v92TRUE_ukbset)

In [98]:
v94_nomatch = diffList(all3979,v94TRUE_ukbset)

In [99]:
v96_nomatch = diffList(all3979,v96TRUE_ukbset)

In [100]:
v97_nomatch = diffList(all3979,v97TRUE_ukbset)

In [101]:
# what uniprot IDs are not in any of the releases as 100% identical to canonical
shared = set(v85_nomatch) & set(v92_nomatch) & set(v94_nomatch) & set(v96_nomatch) & set(v97_nomatch)
len(shared)

49

In [105]:
shared = list(shared)
# these IDs should not be in the True files from each release at all 
shared

['P07686',
 'Q9P2N6',
 'P11532',
 'Q9BX63',
 'Q8WX93',
 'Q68E01',
 'Q96ME1',
 'O15392',
 'Q9BV68',
 'O75400',
 'Q14135',
 'Q14558',
 'Q6PKG0',
 'O43708',
 'Q03001',
 'P16278',
 'O14965',
 'Q15170',
 'Q9Y679',
 'Q9Y520',
 'Q12912',
 'P36639',
 'P30837',
 'P02765',
 'P53990',
 'A6NNF4',
 'Q9BZ29',
 'O15061',
 'P20839',
 'P17927',
 'P11182',
 'Q9UMY4',
 'Q8NBT2',
 'A0FGR8',
 'O60645',
 'Q9UNH6',
 'Q9UNH7',
 'Q8WWI1',
 'Q8NBJ7',
 'Q8NCA5',
 'Q9UJ41',
 'Q8IY17',
 'P11586',
 'Q99729',
 'Q9NRG7',
 'Q9UM54',
 'Q13459',
 'Q86TG7',
 'P18887']

In [108]:
# Checking if shared uniprot ID (not in True release set) exists in list  
# QC
def canonicalMatchNotFound(els, sharedls):
    for id in sharedls:
        ukbid = id
        if ukbid in els:
            print ("Element Exists")
    print("done checking list")

In [110]:
canonicalMatchNotFound(v85TRUE_ukbset, shared)
canonicalMatchNotFound(v92TRUE_ukbset, shared)
canonicalMatchNotFound(v94TRUE_ukbset, shared)
canonicalMatchNotFound(v96TRUE_ukbset, shared)
canonicalMatchNotFound(v97TRUE_ukbset, shared)

done checking list
done checking list
done checking list
done checking list
done checking list


In [111]:
shared


['P07686',
 'Q9P2N6',
 'P11532',
 'Q9BX63',
 'Q8WX93',
 'Q68E01',
 'Q96ME1',
 'O15392',
 'Q9BV68',
 'O75400',
 'Q14135',
 'Q14558',
 'Q6PKG0',
 'O43708',
 'Q03001',
 'P16278',
 'O14965',
 'Q15170',
 'Q9Y679',
 'Q9Y520',
 'Q12912',
 'P36639',
 'P30837',
 'P02765',
 'P53990',
 'A6NNF4',
 'Q9BZ29',
 'O15061',
 'P20839',
 'P17927',
 'P11182',
 'Q9UMY4',
 'Q8NBT2',
 'A0FGR8',
 'O60645',
 'Q9UNH6',
 'Q9UNH7',
 'Q8WWI1',
 'Q8NBJ7',
 'Q8NCA5',
 'Q9UJ41',
 'Q8IY17',
 'P11586',
 'Q99729',
 'Q9NRG7',
 'Q9UM54',
 'Q13459',
 'Q86TG7',
 'P18887']

## testing 1 Uniprot ID that is 'notin97' release fasta

In [124]:
ucan = 'MAAVRMLRTWSRNAGKLICVRYFQTCGNVHVLKPNYVCFFGYPSFKYSHPHHFLKTTAALRGQVVQFKLSDIGEGIREVTVKEWYVKEGDTVSQFDSICEVQSDKASVTITSRYDGVIKKLYYNLDDIAYVGKPLVDIETEALKDSEEDVVETPAVSHDEHTHQEIKGRKTLATPAVRRLAMENNIKLSEVVGSGKDGRILKEDILNYLEKQTGAILPPSPKVEIMPPPPKPKDMTVPILVSKPPVFTGKDKTEPIKGFQKAMVKTMSAALKIPHFGYCDEIDLTELVKLREELKPIAFARGIKLSFMPFFLKAASLGLLQFPILNASVDENCQNITYKASHNIGIAMDTEQGLIVPNVKNVQICSIFDIATELNRLQKLGSVGQLSTTDLTGGTFTLSNIGSIGGTFAKPVIMPPEVAIGALGSIKAIPRFNQKGEVYKAQIMNVSWSADHRVIDGATMSRFSNLWKSYLENPAFMLLDLK'
print(len(ucan))

482


In [125]:
P11182_v97 = "MAAVRMLRTWSRNAGKLICVRYFQTCGNVHVLKPNYVCFFGYPSFKYSHPHHFLKTTAALRGQVVQFKLSDIGEGIREVTVKEWYVKEGDTVSQFDSICEVQSDKASVTITSRYDGVIKKLYYNLDDIAYVGKPLVDIETEALKDSEEDVVETPAVSHDEHTHQEIKGRKTLATPAVRRLAMENNIKLSEVVGSGKDGRILKEDILNYLEKQTGAILPPSPKVEIMPPPPKPKDMTVPILVSKPPVFTGKDKTEPIKGFQKAMVKTMSAALKIPHFGYCDEIDLTELVKLREELKPIAFARGIKLSFMPFFLKAASLGLLQFPILNASVDENCQNITYKASHNIGIAMDTEQGLIVPNVKNVQICSIFDIATELNRLQKLGSVSQLSTTDLTGGTFTLSNIGSIGGTFAKPVIMPPEVAIGALGSIKAIPRFNQKGEVYKAQIMNVSWSADHRVIDGATMSRFSNLWKSYLENPAFMLLDLK"

P11182uniprot = "MAAVRMLRTWSRNAGKLICVRYFQTCGNVHVLKPNYVCFFGYPSFKYSHPHHFLKTTAALRGQVVQFKLSDIGEGIREVTVKEWYVKEGDTVSQFDSICEVQSDKASVTITSRYDGVIKKLYYNLDDIAYVGKPLVDIETEALKDSEEDVVETPAVSHDEHTHQEIKGRKTLATPAVRRLAMENNIKLSEVVGSGKDGRILKEDILNYLEKQTGAILPPSPKVEIMPPPPKPKDMTVPILVSKPPVFTGKDKTEPIKGFQKAMVKTMSAALKIPHFGYCDEIDLTELVKLREELKPIAFARGIKLSFMPFFLKAASLGLLQFPILNASVDENCQNITYKASHNIGIAMDTEQGLIVPNVKNVQICSIFDIATELNRLQKLGSVGQLSTTDLTGGTFTLSNIGSIGGTFAKPVIMPPEVAIGALGSIKAIPRFNQKGEVYKAQIMNVSWSADHRVIDGATMSRFSNLWKSYLENPAFMLLDLK"

In [126]:
P11182_v97 == P11182uniprot

False

In [127]:
print(len(P11182_v97))

482


In [128]:
print(len(P11182uniprot))

482


In [138]:
import difflib
output_list = [li for li in difflib.ndiff(P11182_v97, P11182uniprot) if li[0] != ' ']
output_list

['- S', '+ G']

# checking from 49 list of uniprot IDs with no canonical match in any releases

In [116]:
test = 'A6NNF4'
v97[v97['UniProtSP_xref']== test]
# not in 97

,ENSPv,ENSP,Length,proSequence,MISMAP_SCORE_ENSP,UniProtSP_xref,Identical_UKB_seq,matchedUKBcanonical,hammingScore,hamNormalizedScore,levenshteinScore
43,ENSP00000432583.2,ENSP00000432583,77,MGLLTFRDVAIEFSLEEWQCLDTAQKNLYRNVMLENYRNLAFLGIA...,True,A6NNF4,False,False,662,0.897019,661
44,ENSP00000471516.1,ENSP00000471516,616,MGLLTFRDVAIEFSLEEWQCLDTAQKNLYRNVMLENYRNLAFLGIA...,True,A6NNF4,False,False,122,0.165312,122


In [117]:
v96[v96['UniProtSP_xref']== test]
# no identical match in 96

,ENSPv,ENSP,Length,proSequence,MISMAP_SCORE_ENSP,UniProtSP_xref,Identical_UKB_seq,matchedUKBcanonical,hammingScore,hamNormalizedScore,levenshteinScore
43,ENSP00000432583.2,ENSP00000432583,77,MGLLTFRDVAIEFSLEEWQCLDTAQKNLYRNVMLENYRNLAFLGIA...,True,A6NNF4,False,False,662,0.897019,661
44,ENSP00000471516.1,ENSP00000471516,616,MGLLTFRDVAIEFSLEEWQCLDTAQKNLYRNVMLENYRNLAFLGIA...,True,A6NNF4,False,False,122,0.165312,122


In [118]:
v94[v94['UniProtSP_xref']== test]

,ENSPv,ENSP,Length,proSequence,MISMAP_SCORE_ENSP,UniProtSP_xref,Identical_UKB_seq,matchedUKBcanonical,hammingScore,hamNormalizedScore,levenshteinScore
43,ENSP00000432583.2,ENSP00000432583,77,MGLLTFRDVAIEFSLEEWQCLDTAQKNLYRNVMLENYRNLAFLGIA...,True,A6NNF4,False,False,662,0.897019,661
44,ENSP00000471516.1,ENSP00000471516,616,MGLLTFRDVAIEFSLEEWQCLDTAQKNLYRNVMLENYRNLAFLGIA...,True,A6NNF4,False,False,122,0.165312,122


In [119]:
v92[v92['UniProtSP_xref']== test]

,ENSPv,ENSP,Length,proSequence,MISMAP_SCORE_ENSP,UniProtSP_xref,Identical_UKB_seq,matchedUKBcanonical,hammingScore,hamNormalizedScore,levenshteinScore
43,ENSP00000432583.2,ENSP00000432583,77,MGLLTFRDVAIEFSLEEWQCLDTAQKNLYRNVMLENYRNLAFLGIA...,True,A6NNF4,False,False,662,0.897019,661
44,ENSP00000471516.1,ENSP00000471516,616,MGLLTFRDVAIEFSLEEWQCLDTAQKNLYRNVMLENYRNLAFLGIA...,True,A6NNF4,False,False,122,0.165312,122


In [120]:
v85[v85['UniProtSP_xref']== test]
# v85 has ENSP with same length but differs by 1 substituion

,ENSPv,ENSP,Length,proSequence,MISMAP_SCORE_ENSP,UniProtSP_xref,Identical_UKB_seq,matchedUKBcanonical,hammingScore,hamNormalizedScore,levenshteinScore
50,ENSP00000317125.7,ENSP00000317125,738,MGLLTFRDVAIEFSLEEWQCLDTAQKNLYRNVMLENYRNLAFLGIA...,True,A6NNF4,False,False,1,0.001355,1
51,ENSP00000432583.2,ENSP00000432583,77,MGLLTFRDVAIEFSLEEWQCLDTAQKNLYRNVMLENYRNLAFLGIA...,True,A6NNF4,False,False,662,0.897019,661


In [136]:
# for A6NNF4 same length confirming 1 substition diff
v85seq = 'MGLLTFRDVAIEFSLEEWQCLDTAQKNLYRNVMLENYRNLAFLGIAVSKPDLIICLEKEKEPWNMKRDEMVDEPPGICPHFAQDIWPEQGVEDSFQKVILRRFEKCGHENLQLRKGCKSVDECKVHKEGYNGLNQCFTTTQGKASQCGKYLKVFYKFINLNRYKIRHTRKKPFKCKNCVKSFCMFSHKTQHKSIYTTEKSYKCKECGKTFNWSSTLTNHKKTHTEEKPYKCEEYGKAFNQSSNYTTHKVTHTGEKPYKCEECGKAFSQSSTLTIHKRIHTGEKPCKCEECGKAFSQPSALTIHKRMHIGEKPYKCEECGKAFVWSSTLTRHKRLHSGEKPYKCEECAKAFSQFGHLTTHRIIHTGEKPYKCEECGKAFIWPSTLTKHKRIHTGEKPYKCEECGKAFHRSSNLTKHKIIHTGEKPYKCEECGKAFIWSSNLTEHKKIHTREKPYKCEECSKAFSRSSALTTHKRMHTGEKPYKCEECGKAFSQSSTLTAHKIIHTGEKPYKCEECGKAFILSSTLSKHKRIHTGEKPYKCEECGKTFNQSSNLSTHKIIHTGEKPYKCEECGKAFNRSSNLSTHKIIHTGEKPYKCDECGKSFIWSSTLFKHKIIHTGEKPYKCEECGKAFNHSQILLHIRHKRMHTGEKPYKCEECGKSFNLSSTFIKHKVIHTGVKLYKCEECGKVFFWSSALTRHKKIHAGQQPYKWEKIGKAFNQSSHLTTDKITHIGEKSYKCE'
uniprotseq = 'MGLLTFRDVAIEFSLEEWQCLDTAQKNLYRNVMLENYRNLAFLGIAVSKPDLIICLEKEKEPWNMKRDEMVDEPPGICPHFAQDIWPEQGVEDSFQKVILRRFEKCGHENLQLRKGCKSVDECKVHKEGYNGLNQCFTTTQGKASQCGKYLKVFYKFINLNRYKIRHTRKKPFKCKNCVKSFCMFSHKTQHKSIYTTEKSYKCKECGKTFNWSSTLTNHKKTHTEEKPYKCEEYGKAFNQSSNYTTHKVTHTGEKPYKCEECGKAFSQSSTLTIHKRIHTGEKPCKCEECGKAFSQPSALTIHKRMHIGEKPYKCEECGKAFVWSSTLTRHKRLHSGEKPYKCEECAKAFSQFGHLTTHRIIHTGEKPYKCEECGKAFIWPSTLTKHKRIHTGEKPYKCEECGKAFHRSSNLTKHKIIHTGEKPYKCEECGKAFIWSSNLTEHKKIHTREKPYKCEECSKAFSRSSALTTHKRMHTGEKPYKCEECGKAFSQSSTLTAHKIIHTGEKPYKCEECGKAFILSSTLSKHKRIHTGEKPYKCEECGKTFNQSSNLSTHKIIHTGEKPYKCEECGKAFNRSSNLSTHKIIHTGEKPYKCDECGKSFIWSSTLFKHKRIHTGEKPYKCEECGKAFNHSQILLHIRHKRMHTGEKPYKCEECGKSFNLSSTFIKHKVIHTGVKLYKCEECGKVFFWSSALTRHKKIHAGQQPYKWEKIGKAFNQSSHLTTDKITHIGEKSYKCE'

```bash
grep A6NNF4 MISMAP_SCORE_UniprotFastaCKabund_w_UniprotCYSLYSpositionsLabeled_3979.csv 
3664,A6NNF4,738,MGLLTFRDVAIEFSLEEWQCLDTAQKNLYRNVMLENYRNLAFLGIAVSKPDLIICLEKEKEPWNMKRDEMVDEPPGICPHFAQDIWPEQGVEDSFQKVILRRFEKCGHENLQLRKGCKSVDECKVHKEGYNGLNQCFTTTQGKASQCGKYLKVFYKFINLNRYKIRHTRKKPFKCKNCVKSFCMFSHKTQHKSIYTTEKSYKCKECGKTFNWSSTLTNHKKTHTEEKPYKCEEYGKAFNQSSNYTTHKVTHTGEKPYKCEECGKAFSQSSTLTIHKRIHTGEKPCKCEECGKAFSQPSALTIHKRMHIGEKPYKCEECGKAFVWSSTLTRHKRLHSGEKPYKCEECAKAFSQFGHLTTHRIIHTGEKPYKCEECGKAFIWPSTLTKHKRIHTGEKPYKCEECGKAFHRSSNLTKHKIIHTGEKPYKCEECGKAFIWSSNLTEHKKIHTREKPYKCEECSKAFSRSSALTTHKRMHTGEKPYKCEECGKAFSQSSTLTAHKIIHTGEKPYKCEECGKAFILSSTLSKHKRIHTGEKPYKCEECGKTFNQSSNLSTHKIIHTGEKPYKCEECGKAFNRSSNLSTHKIIHTGEKPYKCDECGKSFIWSSTLFKHKRIHTGEKPYKCEECGKAFNHSQILLHIRHKRMHTGEKPYKCEECGKSFNLSSTFIKHKVIHTGVKLYKCEECGKVFFWSSALTRHKKIHAGQQPYKWEKIGKAFNQSSHLTTDKITHIGEKSYKCE,0.065,0.1477,True,1,{55: 'C'}
```

In [137]:
import difflib
output_list = [li for li in difflib.ndiff(v85seq, uniprotseq) if li[0] != ' ']
output_list
# difference between the 2 sequences is I changed to R in uniprot seq

['- I', '+ R']

# SEE 19_09_12_QC_MISMAP_RESULTS for more quality testing on sequence comparison results

---
---
---

---
---
---

---
---
---

---
---
---

# BELOW CODE is MISMAP SCORING!!!

## making ultimate uniprot reference file
```python
truetarget2 = truetarget[['entry','labeled_pos_count','pos_dict']].copy()
truetarget2.columns = ['UniProtSP_xref','total_targets','pos_dict']
print(truetarget2.shape)
truetarget2.head(3)

merging true targets and reference to make ultimate uniprot SOURCE key
refuniprot = pd.merge(reference,truetarget2,how='inner',on=['UniProtSP_xref'])

refuniprot.to_csv("MISMAP_SCORE_UniprotFastaCKabund_w_UniprotCYSLYSpositionsLabeled_3979.csv")
```

In [146]:
reference = pd.read_csv('MISMAP_SCORE_UniprotFastaCKabund_w_UniprotCYSLYSpositionsLabeled_3979.csv')
print(reference.shape)
print(reference.dtypes)
print(reference.columns)
reference.drop(['Unnamed: 0'],inplace=True,axis=1)
reference.drop(['in3979xref'],inplace=True,axis=1)
reference.head(4)

(3979, 9)
Unnamed: 0          int64
UniProtSP_xref     object
UniProt_length      int64
proSequence        object
C_abundance       float64
K_abundance       float64
in3979xref           bool
total_targets       int64
pos_dict           object
dtype: object
Index(['Unnamed: 0', 'UniProtSP_xref', 'UniProt_length', 'proSequence',
       'C_abundance', 'K_abundance', 'in3979xref', 'total_targets',
       'pos_dict'],
      dtype='object')


,UniProtSP_xref,UniProt_length,proSequence,C_abundance,K_abundance,total_targets,pos_dict
0,Q9HAS0,396,MLPSLQESMDGDEKELESSEEGGSAEERRLEPPSSSHYCLYSYRGS...,0.0303,0.0581,1,{182: 'C'}
1,Q86X76,327,MLGFITRPPHRFLSLLCPGLRIPQLSVLCAQPRPRAMAISSSSCEL...,0.0459,0.0245,3,"{161: 'K', 165: 'C', 203: 'C'}"
2,Q9NQR4,276,MTSFRLALIQLQISSIKSDNVTRACSFIREAATQGAKIVSLPECFN...,0.0254,0.0652,7,"{44: 'C', 52: 'K', 123: 'K', 130: 'K', 146: 'C..."
3,Q9Y314,301,MTRHGKNCTAGAVYTYHEKKKDTAASGYGTQNIRLSRDAVKDFDCC...,0.0299,0.0797,3,"{8: 'C', 78: 'K', 185: 'C'}"


In [147]:
#refuniprot.columns = ['UniProtSP_xref', 'UniProt_length', 'proSequence', 'C_abundance', 'K_abundance',
      # 'in3979xref', 'total_targets', 'pos_dict']
# simplifying df

# droppign the columns i dont need, like proSequence and in3979xref
UKB = reference[['UniProtSP_xref', 'UniProt_length','C_abundance', 'K_abundance', 'total_targets', 'pos_dict']].copy()
UKB.head(3)

,UniProtSP_xref,UniProt_length,C_abundance,K_abundance,total_targets,pos_dict
0,Q9HAS0,396,0.0303,0.0581,1,{182: 'C'}
1,Q86X76,327,0.0459,0.0245,3,"{161: 'K', 165: 'C', 203: 'C'}"
2,Q9NQR4,276,0.0254,0.0652,7,"{44: 'C', 52: 'K', 123: 'K', 130: 'K', 146: 'C..."


In [151]:
def mismap_score(enspdf, uniprot):
    ensp_score_file = []
    # merge two df
    mer = pd.merge(enspdf,uniprot,how='inner',on=['UniProtSP_xref'])
    # QC merge
    print('shape ORIGINAL ENSEMBL df: ', enspdf.shape)
    print('shape merge: ',mer.shape)
    print('shape ORIGINAL KEY ref: ',uniprot.shape)
    print()
    for index, row in mer.iterrows():
        newrow = []
        ukb_id = row['UniProtSP_xref']
        ensp = row['ENSP']
        ensp_len = row['Length']
        ukb_len = row['UniProt_length']
        C_abun = row['C_abundance']
        K_abun = row['K_abundance']
        identical = row['Identical_UKB_seq']
        tot_tar = row['total_targets']
        pep = row['proSequence']
        posCK = row['pos_dict']
        
        # col order
        newrow.append(ensp) # 0
        newrow.append(ukb_id) # 1
        newrow.append(identical) # 2
        newrow.append(ukb_len) # 3
        newrow.append(ensp_len) # 4
        newrow.append(C_abun) # 5
        newrow.append(K_abun) # 6
        newrow.append(tot_tar) # 7
        
        # positions found counter
        count = 0
        
        # evaluate as dictionary
        python_dict = literal_eval(posCK) 
        # loop thru position dict
        for key in python_dict:
            k = int(key)
            # convert pos # to index 0-based
            i = k - 1
            # if index within ensembl protein string
            if i < int(ensp_len):
                # get AA from peptide string
                AA = pep[i]
                # get AA from pos dic
                checker = python_dict[key]
                # match, increase count
                if AA == checker:
                    count += 1    
        # fraction correct pos
        cor_per = count/tot_tar
        miss = tot_tar - count
        # fraction missed
        miss_per = miss/tot_tar
        
        # add columns
        newrow.append(count) # 8 
        newrow.append(miss) # 9
        newrow.append(round(cor_per,2)) # 10
        newrow.append(round(miss_per,2)) # 11
        ensp_score_file.append(newrow) # 12
        
    # turn list of list into dataframe
    df = pd.DataFrame(ensp_score_file)
    df.columns = ['ENSP', 'UniProtID', 'my_identical2UKB_score','len_uniprot','len_ensp', 'C_abun_ukb','K_abun_ukb','total_targets','found_count','missed_count','correct_frac','missed_frac']
    return df

In [152]:
df85 = mismap_score(v85, UKB)
df92 = mismap_score(v92, UKB)
df94 = mismap_score(v94, UKB)
df96 = mismap_score(v96, UKB)
df97 = mismap_score(v97, UKB)

shape ORIGINAL ENSEMBL df:  (10272, 11)
shape merge:  (10272, 16)
shape ORIGINAL KEY ref:  (3979, 6)

shape ORIGINAL ENSEMBL df:  (10479, 11)
shape merge:  (10479, 16)
shape ORIGINAL KEY ref:  (3979, 6)

shape ORIGINAL ENSEMBL df:  (10699, 11)
shape merge:  (10699, 16)
shape ORIGINAL KEY ref:  (3979, 6)

shape ORIGINAL ENSEMBL df:  (10750, 11)
shape merge:  (10750, 16)
shape ORIGINAL KEY ref:  (3979, 6)

shape ORIGINAL ENSEMBL df:  (10650, 11)
shape merge:  (10650, 16)
shape ORIGINAL KEY ref:  (3979, 6)



In [155]:
df85.to_csv("MISMAP_SCORED_v85_ENSP_posDict_checked_10272.csv",index=False)
df92.to_csv("MISMAP_SCORED_v92_ENSP_posDict_checked_10479.csv",index=False)
df94.to_csv("MISMAP_SCORED_v94_ENSP_posDict_checked_10699.csv",index=False)
df96.to_csv("MISMAP_SCORED_v96_ENSP_posDict_checked_10750.csv",index=False)
df97.to_csv("MISMAP_SCORED_v97_ENSP_posDict_checked_10650.csv",index=False)

In [157]:
df85.head(5)

,ENSP,UniProtID,my_identical2UKB_score,len_uniprot,len_ensp,C_abun_ukb,K_abun_ukb,total_targets,found_count,missed_count,correct_frac,missed_frac
0,ENSP00000313454,A0AVT1,True,1052,1052,0.0219,0.0722,11,11,0,1.00,0.00
1,ENSP00000399234,A0AVT1,False,1052,389,0.0219,0.0722,11,2,9,0.18,0.82
2,ENSP00000251527,A0FGR8,False,921,893,0.0098,0.0565,1,0,1,0.00,1.00
3,ENSP00000279907,A0JNW5,True,1464,1464,0.0171,0.0710,1,1,0,1.00,0.00
4,ENSP00000349285,A0JNW5,False,1464,522,0.0171,0.0710,1,0,1,0.00,1.00


In [160]:
# saving fraction of ENSP IDs that were not identical to the uniprot sequences
df85FALSE = df85[df85['my_identical2UKB_score'] == False]
df85FALSE.reset_index(drop=True,inplace=True)
print(df85FALSE.shape)

df92FALSE = df92[df92['my_identical2UKB_score'] == False]
df92FALSE.reset_index(drop=True,inplace=True)
print(df92FALSE.shape)

df94FALSE = df94[df94['my_identical2UKB_score'] == False]
df94FALSE.reset_index(drop=True,inplace=True)
print(df94FALSE.shape)

df96FALSE = df96[df96['my_identical2UKB_score'] == False]
df96FALSE.reset_index(drop=True,inplace=True)
print(df96FALSE.shape)

df97FALSE = df97[df97['my_identical2UKB_score'] == False]
df97FALSE.reset_index(drop=True,inplace=True)
print(df97FALSE.shape)

(4347, 12)
(4522, 12)
(4602, 12)
(4623, 12)
(4598, 12)


In [173]:
reference.shape

(3979, 7)

In [219]:
print(len(set(df85FALSE['UniProtID'])))
print(len(set(df92FALSE['UniProtID'])))
print(len(set(df94FALSE['UniProtID'])))
print(len(set(df96FALSE['UniProtID'])))
print(len(set(df97FALSE['UniProtID'])))

1909
2116
2116
2124
2123


In [168]:
df85FALSE.to_csv("MISMAP_SCORED_differentNumUKBID_1909_v85_notTrue4myIdentityScore_4347.csv",index=False)
df92FALSE.to_csv("MISMAP_SCORED_differentNumUKBID_2116_v92_notTrue4myIdentityScore_4522.csv",index=False)
df94FALSE.to_csv("MISMAP_SCORED_differentNumUKBID_2116_v94_notTrue4myIdentityScore_4602.csv",index=False)
df96FALSE.to_csv("MISMAP_SCORED_differentNumUKBID_2124_v96_notTrue4myIdentityScore_4623.csv",index=False)
df97FALSE.to_csv("MISMAP_SCORED_differentNumUKBID_2123_v97_notTrue4myIdentityScore_4598.csv",index=False)

# controlling for different number of uniprot IDs in each ensembl release file with False identity to canonical sequence score

In [220]:
# for analysis going to make sure all FALSE release files have the same Uniprot IDs linked
f85 = set(df85FALSE['UniProtID'].tolist())
f92 = set(df92FALSE['UniProtID'].tolist())
f94 = set(df94FALSE['UniProtID'].tolist())
f96 = set(df96FALSE['UniProtID'].tolist())
f97 = set(df97FALSE['UniProtID'].tolist())

In [221]:
sharedID = f85 & f92 & f94 & f96 & f97
print(len(sharedID))
print(len(set(sharedID)))
sharedID = list(sharedID)

1805
1805


In [222]:
def addcolumnconditional(mapList, df, dfcol, newcol): 
    mendel = []
    for g in df[dfcol]:
        if g in mapList: 
            mendel.append("True")
        else:
            mendel.append("False")
    df.loc[:,newcol] = mendel
    df.drop(df[df[newcol] == "False"].index, inplace = True)
    df.reset_index(inplace=True, drop=True)
    return df

In [223]:
sameFalse85 = addcolumnconditional(sharedID, df85FALSE, 'UniProtID','UKBIDinFALSEReleases')
sameFalse92 = addcolumnconditional(sharedID, df92FALSE, 'UniProtID','UKBIDinFALSEReleases')
sameFalse94 = addcolumnconditional(sharedID, df94FALSE, 'UniProtID','UKBIDinFALSEReleases')
sameFalse96 = addcolumnconditional(sharedID, df96FALSE, 'UniProtID','UKBIDinFALSEReleases')
sameFalse97 = addcolumnconditional(sharedID, df97FALSE, 'UniProtID','UKBIDinFALSEReleases')

/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#inde

In [224]:
print(sameFalse85.shape)
print(sameFalse92.shape)
print(sameFalse94.shape)
print(sameFalse96.shape)
print(sameFalse97.shape)

(4212, 13)
(4098, 13)
(4173, 13)
(4192, 13)
(4169, 13)


In [225]:
print(len(set(df85FALSE['UniProtID'])))
print(len(set(df92FALSE['UniProtID'])))
print(len(set(df94FALSE['UniProtID'])))
print(len(set(df96FALSE['UniProtID'])))
print(len(set(df97FALSE['UniProtID'])))

1805
1805
1805
1805
1805


In [227]:
sameFalse85.to_csv("MISMAP_SCORED_sameNumUKBID_1805_v85_notTrue4myIdentityScore_4212.csv",index=False)
sameFalse92.to_csv("MISMAP_SCORED_sameNumUKBID_1805_v92_notTrue4myIdentityScore_4098.csv",index=False)
sameFalse94.to_csv("MISMAP_SCORED_sameNumUKBID_1805_v94_notTrue4myIdentityScore_4173.csv",index=False)
sameFalse96.to_csv("MISMAP_SCORED_sameNumUKBID_1805_v96_notTrue4myIdentityScore_4192.csv",index=False)
sameFalse97.to_csv("MISMAP_SCORED_sameNumUKBID_1805_v97_notTrue4myIdentityScore_4169.csv",index=False)

In [169]:
# saving fraction of ENSP IDs that are identical to the uniprot sequences
df85TRUE = df85[df85['my_identical2UKB_score'] == True]
df85TRUE.reset_index(drop=True,inplace=True)
print(df85TRUE.shape)

df92TRUE = df92[df92['my_identical2UKB_score'] ==  True]
df92TRUE.reset_index(drop=True,inplace=True)
print(df92TRUE.shape)

df94TRUE = df94[df94['my_identical2UKB_score'] ==  True]
df94TRUE.reset_index(drop=True,inplace=True)
print(df94TRUE.shape)

df96TRUE = df96[df96['my_identical2UKB_score'] ==  True]
df96TRUE.reset_index(drop=True,inplace=True)
print(df96TRUE.shape)

df97TRUE = df97[df97['my_identical2UKB_score'] ==  True]
df97TRUE.reset_index(drop=True,inplace=True)
print(df97TRUE.shape)

(5925, 12)
(5957, 12)
(6097, 12)
(6127, 12)
(6052, 12)


In [171]:
print(len(set(df85TRUE['UniProtID'])))
print(len(set(df92TRUE['UniProtID'])))
print(len(set(df94TRUE['UniProtID'])))
print(len(set(df96TRUE['UniProtID'])))
print(len(set(df97TRUE['UniProtID'])))

3884
3878
3895
3890
3885


In [174]:
df85TRUE.to_csv("MISMAP_SCORED_differentNumUKBID_3884_v85_True4myIdentityScore_5925.csv",index=False)
df92TRUE.to_csv("MISMAP_SCORED_differentNumUKBID_3878_v92_True4myIdentityScore_5957.csv",index=False)
df94TRUE.to_csv("MISMAP_SCORED_differentNumUKBID_3895_v94_True4myIdentityScore_6097.csv",index=False)
df96TRUE.to_csv("MISMAP_SCORED_differentNumUKBID_3890_v96_True4myIdentityScore_6127.csv",index=False)
df97TRUE.to_csv("MISMAP_SCORED_differentNumUKBID_3885_v97_True4myIdentityScore_6052.csv",index=False)

# 2 versions of score
a. includes all ENSP IDs linked to Uniprot ID

b. includes only ENSP that had pep seq not equal to uniprot pep seq ( these include same number of uniprot IDs 1805)

### score can be calculated 2 ways
- total correct pos/ total counted for uniprot ID
- total missed pos/ total counted for uniprot ID

In [175]:
des_df85 = df85.describe()
des_df92 = df92.describe()
des_df94 = df94.describe()
des_df96 = df96.describe()
des_df97 = df97.describe()
des_df85.to_csv("MISMAP_SCORED_described_10272.csv")
des_df92.to_csv("MISMAP_SCORED_described_10479.csv")
des_df94.to_csv("MISMAP_SCORED_described_10699.csv")
des_df96.to_csv("MISMAP_SCORED_described_10750.csv")
des_df97.to_csv("MISMAP_SCORED_described_10650.csv")

In [230]:
grdf85 = df85.groupby(['UniProtID'])['total_targets','found_count','missed_count'].sum().reset_index()
grdf92 = df92.groupby(['UniProtID'])['total_targets','found_count','missed_count'].sum().reset_index()
grdf94 = df94.groupby(['UniProtID'])['total_targets','found_count','missed_count'].sum().reset_index()
grdf96 = df96.groupby(['UniProtID'])['total_targets','found_count','missed_count'].sum().reset_index()
grdf97 = df97.groupby(['UniProtID'])['total_targets','found_count','missed_count'].sum().reset_index()

In [231]:
grdf85

,UniProtID,total_targets,found_count,missed_count
0,A0AVT1,22,13,9
1,A0FGR8,1,0,1
2,A0JNW5,3,1,2
3,A0MZ66,4,2,2
4,A1KXE4,2,2,0
5,A1L0T0,1,1,0
6,A2A288,9,7,2
7,A2RUC4,1,1,0
8,A3KN83,4,2,2
9,A4D1E9,3,1,2


# comparing all release files FALSE for matching canonical sequence, there are 1805 uniprot IDs shared between all release files

In [232]:
grsameFalse85 = sameFalse85.groupby(['UniProtID'])['total_targets','found_count','missed_count'].sum().reset_index()
grsameFalse92 = sameFalse92.groupby(['UniProtID'])['total_targets','found_count','missed_count'].sum().reset_index()
grsameFalse94 = sameFalse94.groupby(['UniProtID'])['total_targets','found_count','missed_count'].sum().reset_index()
grsameFalse96 = sameFalse96.groupby(['UniProtID'])['total_targets','found_count','missed_count'].sum().reset_index()
grsameFalse97 = sameFalse97.groupby(['UniProtID'])['total_targets','found_count','missed_count'].sum().reset_index()

In [233]:
grsameFalse85

,UniProtID,total_targets,found_count,missed_count
0,A0AVT1,11,2,9
1,A0FGR8,1,0,1
2,A0JNW5,2,0,2
3,A0MZ66,3,1,2
4,A3KN83,2,0,2
5,A4D1E9,2,0,2
6,A4D1F6,1,0,1
7,A5YKK6,9,3,6
8,A6NC98,2,0,2
9,A6NHL2,6,0,6


## Results, not all Uniprot IDs have a linked ENSP with 100% seq. identity

### about 56% of the release score ENSP IDs are 100% identical to UKB

In [234]:
dflist = [grdf85,grdf92,grdf94,grdf96,grdf97]
for i in dflist:
    print(describeMe(i))

(3979, 4)
Index(['UniProtID', 'total_targets', 'found_count', 'missed_count'], dtype='object')
UniProtID        0
total_targets    0
found_count      0
missed_count     0
dtype: int64
None
(3979, 4)
Index(['UniProtID', 'total_targets', 'found_count', 'missed_count'], dtype='object')
UniProtID        0
total_targets    0
found_count      0
missed_count     0
dtype: int64
None
(3979, 4)
Index(['UniProtID', 'total_targets', 'found_count', 'missed_count'], dtype='object')
UniProtID        0
total_targets    0
found_count      0
missed_count     0
dtype: int64
None
(3979, 4)
Index(['UniProtID', 'total_targets', 'found_count', 'missed_count'], dtype='object')
UniProtID        0
total_targets    0
found_count      0
missed_count     0
dtype: int64
None
(3979, 4)
Index(['UniProtID', 'total_targets', 'found_count', 'missed_count'], dtype='object')
UniProtID        0
total_targets    0
found_count      0
missed_count     0
dtype: int64
None


In [235]:
dflist = [grsameFalse85,grsameFalse92,grsameFalse94,grsameFalse96,grsameFalse97]
for i in dflist:
    print(describeMe(i))

(1805, 4)
Index(['UniProtID', 'total_targets', 'found_count', 'missed_count'], dtype='object')
UniProtID        0
total_targets    0
found_count      0
missed_count     0
dtype: int64
None
(1805, 4)
Index(['UniProtID', 'total_targets', 'found_count', 'missed_count'], dtype='object')
UniProtID        0
total_targets    0
found_count      0
missed_count     0
dtype: int64
None
(1805, 4)
Index(['UniProtID', 'total_targets', 'found_count', 'missed_count'], dtype='object')
UniProtID        0
total_targets    0
found_count      0
missed_count     0
dtype: int64
None
(1805, 4)
Index(['UniProtID', 'total_targets', 'found_count', 'missed_count'], dtype='object')
UniProtID        0
total_targets    0
found_count      0
missed_count     0
dtype: int64
None
(1805, 4)
Index(['UniProtID', 'total_targets', 'found_count', 'missed_count'], dtype='object')
UniProtID        0
total_targets    0
found_count      0
missed_count     0
dtype: int64
None


# adding score col to df's that include all identity True and only identity False rows (with same # of uniprot IDs)
### all df's have 3979 uniprot IDs but different numbers of ENSP IDs
### only False df's have 1805 uniprot IDs and different number of ENSP IDs


In [236]:
def divideCol(df):
    df['frac_found'] = df['found_count']/df['total_targets']
    df['frac_missed'] = df['missed_count']/df['total_targets']
    return df

In [237]:
# 3979 uniprot IDs
frac_85 = divideCol(grdf85)
frac_92 = divideCol(grdf92)
frac_94 = divideCol(grdf94)
frac_96 = divideCol(grdf96)
frac_97 = divideCol(grdf97)

In [238]:
frac_85.head(5)

,UniProtID,total_targets,found_count,missed_count,frac_found,frac_missed
0,A0AVT1,22,13,9,0.590909,0.409091
1,A0FGR8,1,0,1,0.000000,1.000000
2,A0JNW5,3,1,2,0.333333,0.666667
3,A0MZ66,4,2,2,0.500000,0.500000
4,A1KXE4,2,2,0,1.000000,0.000000


In [239]:
fracfal_85 = divideCol(grsameFalse85)
fracfal_92 = divideCol(grsameFalse92)
fracfal_94 = divideCol(grsameFalse94)
fracfal_96 = divideCol(grsameFalse96)
fracfal_97 = divideCol(grsameFalse97)

In [240]:
print(fracfal_85.shape)
print(fracfal_92.shape)
print(fracfal_94.shape)
print(fracfal_96.shape)
print(fracfal_97.shape)

(1805, 6)
(1805, 6)
(1805, 6)
(1805, 6)
(1805, 6)


In [241]:
# saving grouped scores, all rows including identical matches to uniprot canonical sequence
frac_85.to_csv("groupedMISMAP_score_85_3979.csv",index=False)
frac_92.to_csv("groupedMISMAP_score_92_3979.csv",index=False)
frac_94.to_csv("groupedMISMAP_score_94_3979.csv",index=False)
frac_96.to_csv("groupedMISMAP_score_96_3979.csv",index=False)
frac_97.to_csv("groupedMISMAP_score_97_3979.csv",index=False)

# saving grouped scores, only rows that were not identical to uniprot canonical sequence
fracfal_85.to_csv("groupedMISMAP_score_85_FALSEidentity_1805.csv",index=False)
fracfal_92.to_csv("groupedMISMAP_score_92_FALSEidentity_1805.csv",index=False)
fracfal_94.to_csv("groupedMISMAP_score_94_FALSEidentity_1805.csv",index=False)
fracfal_96.to_csv("groupedMISMAP_score_96_FALSEidentity_1805.csv",index=False)
fracfal_97.to_csv("groupedMISMAP_score_97_FALSEidentity_1805.csv",index=False)

In [242]:
# for R figures
os.chdir("/Users/mariapalafox/Box Sync/CODE_DATA/dir_MAPpaper/R_figures")
print(os.listdir())

['groupedMISMAP_score_85_FALSEidentity_1805.csv', 'Rmerge_MISMAP_ALLIdentityOnly_slope_change_labels_3979.csv', '.DS_Store', 'lines_ALLidentity.pdf', 'Rmerge_MISMAP_ALLIdentityOnly_slope_changing_only_908IDs_4540.csv', 'groupedMISMAP_score_97_3979.csv', 'groupedMISMAP_score_96_3979.csv', 'pyconvert_Rfriendly.py', '19_09_09_RMISMAP_SCORE.Rmd', 'lines_FALSEidentity_changingSlope.pdf', 'lines_FALSEidentity.pdf', 'groupedMISMAP_score_92_3979.csv', 'Rmerge_MISMAP_FalseIdentityOnly_slope_change_labels_1805.csv', 'groupedMISMAP_score_96_FALSEidentity_1805.csv', 'MISMAP_SCORE_UniprotCanonical_3979entries_valueCounts_fromDict.csv', 'box_FALSEidentity_changingSlope.pdf', 'groupedMISMAP_score_85_3979.csv', 'box_FALSEidentity.pdf', 'groupedMISMAP_score_92_FALSEidentity_1805.csv', 'Rmerge_MISMAP_FalseIdentityOnly_slope_changing_only_282IDs_1410.csv', 'MISMAP_SCORE_allScored_described_df85.csv', 'lines_ALLidentity_changingSlope.pdf', 'box_ALLidentity.pdf', 'groupedMISMAP_score_94_FALSEidentity_1805.

In [246]:
# converted files into Rfigure friendly format 
# from pyconvert_Rfriendly.py

import pandas as pd 
import numpy as np 
import csv

# converting .csv files into R figure format for box plots
# x axis is diff releases 
# y axis is fraction missed or found score

def combineDFS(optio):
    # both true and false identity scores
    if optio == "all":
        df85 = pd.read_csv("groupedMISMAP_score_85_3979.csv")
        df92 = pd.read_csv("groupedMISMAP_score_92_3979.csv")
        df94 = pd.read_csv("groupedMISMAP_score_94_3979.csv")
        df96 = pd.read_csv("groupedMISMAP_score_96_3979.csv")
        df97 = pd.read_csv("groupedMISMAP_score_97_3979.csv")

        df85['release'] = 85
        df92['release'] = 92
        df94['release'] = 94
        df96['release'] = 96
        df97['release'] = 97

        # combining dfs on axis 
        df_merge = pd.concat([df85, df92, df94, df96, df97])
        print(df_merge.head(10))
        print(df_merge.tail(10))
        print(df_merge.shape)
        print("all identity scores len: ", 3979*5)

        df_merge.to_csv("Rmerge_MISMAP_AllIdentityOnly_3979.csv",index=False)

    if optio == "false":
        #only false
        df85 = pd.read_csv("groupedMISMAP_score_85_FALSEidentity_1805.csv")
        df92 = pd.read_csv("groupedMISMAP_score_92_FALSEidentity_1805.csv")
        df94 = pd.read_csv("groupedMISMAP_score_94_FALSEidentity_1805.csv")
        df96 = pd.read_csv("groupedMISMAP_score_96_FALSEidentity_1805.csv")
        df97 = pd.read_csv("groupedMISMAP_score_97_FALSEidentity_1805.csv")

        df85['release'] = 85
        df92['release'] = 92
        df94['release'] = 94
        df96['release'] = 96
        df97['release'] = 97

        # combining dfs on axis 
        df_merge = pd.concat([df85, df92, df94, df96, df97])
        print(df_merge.head(10))
        print(df_merge.tail(10))
        print(df_merge.shape)
        print("only false identity scores len: ", 1805*5)

        df_merge.to_csv("Rmerge_MISMAP_FalseIdentityOnly_1805.csv",index=False)

def main():
    combineDFS("all")
    combineDFS("false")
    print("done")
main()


  UniProtID  total_targets  found_count  missed_count  frac_found  \
0    A0AVT1             22           13             9    0.590909   
1    A0FGR8              1            0             1    0.000000   
2    A0JNW5              3            1             2    0.333333   
3    A0MZ66              4            2             2    0.500000   
4    A1KXE4              2            2             0    1.000000   
5    A1L0T0              1            1             0    1.000000   
6    A2A288              9            7             2    0.777778   
7    A2RUC4              1            1             0    1.000000   
8    A3KN83              4            2             2    0.500000   
9    A4D1E9              3            1             2    0.333333   

   frac_missed  release  
0     0.409091       85  
1     1.000000       85  
2     0.666667       85  
3     0.500000       85  
4     0.000000       85  
5     0.000000       85  
6     0.222222       85  
7     0.000000       85  
8     

In [303]:
# merging all the releases together in order to substract out lines with slope 0

merall = "Rmerge_MISMAP_AllIdentityOnly_3979.csv"
merflase = "Rmerge_MISMAP_FalseIdentityOnly_1805.csv"

mall = pd.read_csv(merall)
mfal = pd.read_csv(merflase)

describeMe(mall)
describeMe(mfal)

(19895, 7)
Index(['UniProtID', 'total_targets', 'found_count', 'missed_count',
       'frac_found', 'frac_missed', 'release'],
      dtype='object')
UniProtID        0
total_targets    0
found_count      0
missed_count     0
frac_found       0
frac_missed      0
release          0
dtype: int64
(9025, 7)
Index(['UniProtID', 'total_targets', 'found_count', 'missed_count',
       'frac_found', 'frac_missed', 'release'],
      dtype='object')
UniProtID        0
total_targets    0
found_count      0
missed_count     0
frac_found       0
frac_missed      0
release          0
dtype: int64


In [304]:
mall[mall['UniProtID']=='A0JNW5']

,UniProtID,total_targets,found_count,missed_count,frac_found,frac_missed,release
2,A0JNW5,3,1,2,0.333333,0.666667,85
3981,A0JNW5,2,1,1,0.500000,0.500000,92
7960,A0JNW5,2,1,1,0.500000,0.500000,94
11939,A0JNW5,2,1,1,0.500000,0.500000,96
15918,A0JNW5,2,1,1,0.500000,0.500000,97


In [250]:
# creating list of all identity scores fraction found
allgroupfound = mall.groupby('UniProtID')['frac_found'].apply(list)
allFOUND = pd.DataFrame(allgroupfound,index=None)
allFOUND.reset_index(inplace=True)
allFOUND.head(5)

,UniProtID,frac_found
0,A0AVT1,"[0.5909090909090909, 0.5909090909090909, 0.590..."
1,A0FGR8,"[0.0, 0.0, 0.0, 0.0, 0.0]"
2,A0JNW5,"[0.3333333333333333, 0.5, 0.5, 0.5, 0.5]"
3,A0MZ66,"[0.5, 0.2, 0.2, 0.2, 0.2]"
4,A1KXE4,"[1.0, 1.0, 1.0, 1.0, 1.0]"


In [251]:
# creating list of all identity scores fraction missed
allgroupmiss = mall.groupby('UniProtID')['frac_missed'].apply(list)
allMISSED = pd.DataFrame(allgroupmiss,index=None)
allMISSED.reset_index(inplace=True)
allMISSED.head(5)

,UniProtID,frac_missed
0,A0AVT1,"[0.4090909090909091, 0.4090909090909091, 0.409..."
1,A0FGR8,"[1.0, 1.0, 1.0, 1.0, 1.0]"
2,A0JNW5,"[0.6666666666666666, 0.5, 0.5, 0.5, 0.5]"
3,A0MZ66,"[0.5, 0.8, 0.8, 0.8, 0.8]"
4,A1KXE4,"[0.0, 0.0, 0.0, 0.0, 0.0]"


---


In [253]:
# creating list of false identity scores fraction found
falsegroupfound = mfal.groupby('UniProtID')['frac_found'].apply(list)
falseFOUND = pd.DataFrame(falsegroupfound,index=None)
falseFOUND.reset_index(inplace=True)
falseFOUND.head(5)

,UniProtID,frac_found
0,A0AVT1,"[0.18181818181818185, 0.18181818181818185, 0.1..."
1,A0FGR8,"[0.0, 0.0, 0.0, 0.0, 0.0]"
2,A0JNW5,"[0.0, 0.0, 0.0, 0.0, 0.0]"
3,A0MZ66,"[0.3333333333333333, 0.0, 0.0, 0.0, 0.0]"
4,A3KN83,"[0.0, 0.0, 0.0, 0.0, 0.0]"


In [254]:
# creating list of all identity scores fraction found
falsegroupmissed = mfal.groupby('UniProtID')['frac_missed'].apply(list)
falseMISSED = pd.DataFrame(falsegroupmissed,index=None)
falseMISSED.reset_index(inplace=True)
falseMISSED.head(5)

,UniProtID,frac_missed
0,A0AVT1,"[0.8181818181818182, 0.8181818181818182, 0.818..."
1,A0FGR8,"[1.0, 1.0, 1.0, 1.0, 1.0]"
2,A0JNW5,"[1.0, 1.0, 1.0, 1.0, 1.0]"
3,A0MZ66,"[0.6666666666666666, 1.0, 1.0, 1.0, 1.0]"
4,A3KN83,"[1.0, 1.0, 1.0, 1.0, 1.0]"


In [268]:
# CHECKING IF ALL values in a list are equal
test = [0.3333333333333333, 0.0, 0.0, 0.0, 0.0]
if all(x == test[0] for x in test):
#if(len(set(test))==1):
  print("All elements in list are same")
else:
  print("All elements in list are not same") 


# checking if all values in list are different
testD = [0.3333333333333333, 20.0, 30.0, 40.0, 50.0]
setL = len(set(testD))
if setL == 5:
      print("All elements in list are different")

All elements in list are not same
All elements in list are different


In [262]:
# checks if all values in list are identical
def mismaplines(df, colname):
    strline = []
    for index, row in df.iterrows():
        ukbid = row['UniProtID']
        ls = row[colname]
        # test is all list elem equal
        if all(x == ls[0] for x in ls):
        #if(len(set(ls)) == 1):
            strline.append("True")
        else:
            strline.append("False")
    df.loc[:,'slope0'] = strline
    print(df.shape)
    print(df.head(3))
    return df

In [269]:
falseFOUND_slope = mismaplines(falseFOUND, 'frac_found')
falseMISSED_slope = mismaplines(falseMISSED,'frac_missed')
allFOUND_slope = mismaplines(allFOUND,'frac_found')
allMISSED_slope = mismaplines(allMISSED,'frac_missed')
checkColumnValues(falseFOUND_slope,'slope0')
checkColumnValues(falseMISSED_slope,'slope0')
checkColumnValues(allFOUND_slope,'slope0')
checkColumnValues(allMISSED_slope,'slope0')

(1805, 3)
  UniProtID                                         frac_found slope0
0    A0AVT1  [0.18181818181818185, 0.18181818181818185, 0.1...   True
1    A0FGR8                          [0.0, 0.0, 0.0, 0.0, 0.0]   True
2    A0JNW5                          [0.0, 0.0, 0.0, 0.0, 0.0]   True
(1805, 3)
  UniProtID                                        frac_missed slope0
0    A0AVT1  [0.8181818181818182, 0.8181818181818182, 0.818...   True
1    A0FGR8                          [1.0, 1.0, 1.0, 1.0, 1.0]   True
2    A0JNW5                          [1.0, 1.0, 1.0, 1.0, 1.0]   True
(3979, 3)
  UniProtID                                         frac_found slope0
0    A0AVT1  [0.5909090909090909, 0.5909090909090909, 0.590...   True
1    A0FGR8                          [0.0, 0.0, 0.0, 0.0, 0.0]   True
2    A0JNW5           [0.3333333333333333, 0.5, 0.5, 0.5, 0.5]  False
(3979, 3)
  UniProtID                                        frac_missed slope0
0    A0AVT1  [0.4090909090909091, 0.40909090909090

# straight lines in R figure subtraction

- total 282 lines are not straight in df with false identity only 
- total 908 lines are not straight in df with all identities
- 0.1562326869806094 = 282/1805
- 0.22819803970846947 = 908/3979

In [274]:
# checking if all values are different in list, most variable IDs
def mismaplines_dynamic(df, colname):
    diffline = []
    for index, row in df.iterrows():
        ukbid = row['UniProtID']
        ls = row[colname]
        # test is all list elem equal
        lenLS = len(set(ls))
        if lenLS == 4:
            diffline.append("True")
        else:
            diffline.append("False")
    df.loc[:,'dynamic_slope'] = diffline
    print(df.shape)
    print(df.head(3))
    return df

In [275]:
falseFOUND_dynamic = mismaplines_dynamic(falseFOUND, 'frac_found')
falseMISSED_dynamic = mismaplines_dynamic(falseMISSED,'frac_missed')
allFOUND_dynamic = mismaplines_dynamic(allFOUND,'frac_found')
allMISSED_dynamic = mismaplines_dynamic(allMISSED,'frac_missed')
checkColumnValues(falseFOUND_dynamic,'dynamic_slope')
checkColumnValues(falseMISSED_dynamic,'dynamic_slope')
checkColumnValues(allFOUND_dynamic,'dynamic_slope')
checkColumnValues(allMISSED_dynamic,'dynamic_slope')

(1805, 5)
  UniProtID                                         frac_found slope0  \
0    A0AVT1  [0.18181818181818185, 0.18181818181818185, 0.1...   True   
1    A0FGR8                          [0.0, 0.0, 0.0, 0.0, 0.0]   True   
2    A0JNW5                          [0.0, 0.0, 0.0, 0.0, 0.0]   True   

  dynamicSlope dynamic_slope  
0        False         False  
1        False         False  
2        False         False  
(1805, 5)
  UniProtID                                        frac_missed slope0  \
0    A0AVT1  [0.8181818181818182, 0.8181818181818182, 0.818...   True   
1    A0FGR8                          [1.0, 1.0, 1.0, 1.0, 1.0]   True   
2    A0JNW5                          [1.0, 1.0, 1.0, 1.0, 1.0]   True   

  dynamicSlope dynamic_slope  
0        False         False  
1        False         False  
2        False         False  
(3979, 5)
  UniProtID                                         frac_found slope0  \
0    A0AVT1  [0.5909090909090909, 0.5909090909090909, 0.590... 

# 4 out of 5 different scores= dynamic lines in R figure subtraction

- total 1 lines are dynamic in df with false identity only 
- total 11 lines are dynamic in df with all identities


In [277]:
# droping column for 5 out of 5 different frac_found scores
falseFOUND_dynamic.drop(['dynamicSlope'],inplace=True,axis=1)
falseMISSED_dynamic.drop(['dynamicSlope'],inplace=True,axis=1)
allFOUND_dynamic.drop(['dynamicSlope'],inplace=True,axis=1)
allMISSED_dynamic.drop(['dynamicSlope'],inplace=True,axis=1)

In [297]:
falseMISSED.head(3)

,UniProtID,frac_missed,slope0,dynamic_slope
0,A0AVT1,"[0.8181818181818182, 0.8181818181818182, 0.818...",True,False
1,A0FGR8,"[1.0, 1.0, 1.0, 1.0, 1.0]",True,False
2,A0JNW5,"[1.0, 1.0, 1.0, 1.0, 1.0]",True,False


In [296]:
allMISSED.head(3)

,UniProtID,frac_missed,slope0,dynamic_slope
0,A0AVT1,"[0.4090909090909091, 0.4090909090909091, 0.409...",True,False
1,A0FGR8,"[1.0, 1.0, 1.0, 1.0, 1.0]",True,False
2,A0JNW5,"[0.6666666666666666, 0.5, 0.5, 0.5, 0.5]",False,False


In [291]:
falseMISSED.equals(falseMISSED_dynamic)

True

In [293]:
allMISSED.equals(allMISSED_dynamic)
# the mismap function adds column to original file no need to create new variable with return

True

In [289]:
# saving files grouped in way that allowed for easy slope0 and dynamic scoring
# need to add these labels to R file merge 
falseMISSED.to_csv("R_falseOnlyIdentity_fracMISSED_slope0_dynamic_colnames_1805.csv", index=False)
allMISSED.to_csv("R_allIdentity_fracMISSED_slope0_dynamic_colnames_3979.csv", index=False)

# variables for R figures-use fraction missed

## dynamic = 
* the frac_missed score of each release is different (4 out of 5 unique scores)
* none of the IDs had a different score in all 5 (5 out of 5 unique scores)

## slope 0 = 
* all the frac_missed scores of each release are the same
---
### falseMISSED_dynamic
- 1 out of 1805 uniprot IDs

### allMISSED_dynamic
- 11 out of 3979 uniprot IDs

---
### falseMISSED_slope
- 282 out of 1805 uniprot IDs

### allMISSED_slope
- 908 out of 3979 uniprot IDs


In [294]:
# ID i want to keep because they have a non 0 slope
fms = falseMISSED[falseMISSED['slope0']=='False']
fms_list = fms['UniProtID'].tolist()
ams = allMISSED[allMISSED['slope0']=='False']
ams_list = ams['UniProtID'].tolist()

print(len(fms_list))
print(len(ams_list))

282
908


In [295]:
# ID i want to keep because they have a non 0 slope
fmd = falseMISSED[falseMISSED['dynamic_slope']=='True']
fmd_list = fmd['UniProtID'].tolist()
amd = allMISSED[allMISSED['dynamic_slope']=='True']
amd_list = amd['UniProtID'].tolist()

print(len(fmd_list))
print(len(amd_list))

1
11


## in R merge files,
- since there are 282 unique uniprot IDs from FALSE identity file that are not slope 0, there should be (282*5) rows in merge file labeled
- in ALL merge file there should be 908*5 rows labeled as not slope 0

mall

mfal

In [305]:
mall.head(3)

,UniProtID,total_targets,found_count,missed_count,frac_found,frac_missed,release
0,A0AVT1,22,13,9,0.590909,0.409091,85
1,A0FGR8,1,0,1,0.000000,1.000000,85
2,A0JNW5,3,1,2,0.333333,0.666667,85


In [306]:
mfal.head(3)

,UniProtID,total_targets,found_count,missed_count,frac_found,frac_missed,release
0,A0AVT1,11,2,9,0.181818,0.818182,85
1,A0FGR8,1,0,1,0.000000,1.000000,85
2,A0JNW5,2,0,2,0.000000,1.000000,85


In [307]:
print(mfal.shape)
print(mall.shape)

(9025, 7)
(19895, 7)


In [49]:
def addcolumnconditional(mapList, df, dfcol, newcol): 
    mendel = []
    for g in df[dfcol]:
        if g in mapList: 
            mendel.append("True")
        else:
            mendel.append("False")
    df.loc[:,newcol] = mendel
    return df

In [309]:
false_slope0 = addcolumnconditional(fms_list,mfal,'UniProtID','notSlope0')
all_slope0 = addcolumnconditional(ams_list,mall, 'UniProtID','notSlope0')

In [310]:
print(false_slope0.shape)
print(282*5)
checkColumnValues(false_slope0,'notSlope0')
print()
print()
print(all_slope0.shape)
print(908*5)
checkColumnValues(all_slope0,'notSlope0')

(9025, 8)
1410
  notSlope0  Count
0     False   7615
1      True   1410


(19895, 8)
4540
  notSlope0  Count
0     False  15355
1      True   4540


In [311]:
false_labeled = addcolumnconditional(fmd_list,false_slope0,'UniProtID','dynamicSlope')
all_labeled = addcolumnconditional(amd_list,all_slope0, 'UniProtID','dynamicSlope')

In [312]:
print(false_labeled.shape)
print(1*5)
checkColumnValues(false_labeled,'dynamicSlope')
print()
print()
print(all_labeled.shape)
print(11*5)
checkColumnValues(all_labeled,'dynamicSlope')

(9025, 9)
5
  dynamicSlope  Count
0        False   9020
1         True      5


(19895, 9)
55
  dynamicSlope  Count
0        False  19840
1         True     55


In [315]:
1805 * 5

9025

---
---
---
### the false and all identity file are the lenght of numbers below because i merged files that were already grouped by uniprot ID (condensed all ENSP ID scores into 1 score for uniprot ID)

> 3979 x 5 = 19,895

> 1805 x 5 = 9025

--- 

##### number of slope0 = False in false identity file
282 x 5 = 1410 

##### number of dynamic slope IDs in false identity file
1 x 5 = 5

---

##### number of slope0 = False in all identity file
908 x 5 = 4540

##### number of dynamic slope IDs in all identity file
11 x 5 = 55

**5 for all different releases (85,92,94,96,97)**

In [316]:
false_labeled

,UniProtID,total_targets,found_count,missed_count,frac_found,frac_missed,release,notSlope0,dynamicSlope
0,A0AVT1,11,2,9,0.181818,0.818182,85,False,False
1,A0FGR8,1,0,1,0.000000,1.000000,85,False,False
2,A0JNW5,2,0,2,0.000000,1.000000,85,False,False
3,A0MZ66,3,1,2,0.333333,0.666667,85,True,False
4,A3KN83,2,0,2,0.000000,1.000000,85,False,False
5,A4D1E9,2,0,2,0.000000,1.000000,85,False,False
6,A4D1F6,1,0,1,0.000000,1.000000,85,False,False
7,A5YKK6,9,3,6,0.333333,0.666667,85,False,False
8,A6NC98,2,0,2,0.000000,1.000000,85,False,False
9,A6NHL2,6,0,6,0.000000,1.000000,85,False,False


In [317]:
all_labeled

,UniProtID,total_targets,found_count,missed_count,frac_found,frac_missed,release,notSlope0,dynamicSlope
0,A0AVT1,22,13,9,0.590909,0.409091,85,False,False
1,A0FGR8,1,0,1,0.000000,1.000000,85,False,False
2,A0JNW5,3,1,2,0.333333,0.666667,85,True,False
3,A0MZ66,4,2,2,0.500000,0.500000,85,True,False
4,A1KXE4,2,2,0,1.000000,0.000000,85,False,False
5,A1L0T0,1,1,0,1.000000,0.000000,85,False,False
6,A2A288,9,7,2,0.777778,0.222222,85,True,False
7,A2RUC4,1,1,0,1.000000,0.000000,85,False,False
8,A3KN83,4,2,2,0.500000,0.500000,85,True,False
9,A4D1E9,3,1,2,0.333333,0.666667,85,False,False


In [318]:
# saving labeled merge files
false_labeled.to_csv("Rmerge_MISMAP_SCORED_falseIdentity_1805UKBidsX5_9025rows_allReleases_slope0_dynamic_colnames.csv",index=False)
all_labeled.to_csv("Rmerge_MISMAP_SCORED_allIdentity_3979UKBidsX5_19895rows_allReleases_slope0_dynamic_colnames.csv",index=False)

---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
# RE-DOING dynamic slope score to include several levels of 'dynamic'
- dynamic 1 = all fraction scores in 5 releases match
- dynamic 2 = 2 unique fraction scores in 5 releases
- dynamic 3 = 3 unique ..
- dynamic 4 = 4 unique ..
- dynamic 5 does not exist because no uniprot ID have 5 unique scores for all 5 releases

### Laser figure datasets will have added handle to grab the 49 uniprot IDs that do not have a canonical sequence match in any of the releases

shared = ['P07686',
 'Q9P2N6',
 'P11532',
 'Q9BX63',
 'Q8WX93',
 'Q68E01',
 'Q96ME1',
 'O15392',
 'Q9BV68',
 'O75400',
 'Q14135',
 'Q14558',
 'Q6PKG0',
 'O43708',
 'Q03001',
 'P16278',
 'O14965',
 'Q15170',
 'Q9Y679',
 'Q9Y520',
 'Q12912',
 'P36639',
 'P30837',
 'P02765',
 'P53990',
 'A6NNF4',
 'Q9BZ29',
 'O15061',
 'P20839',
 'P17927',
 'P11182',
 'Q9UMY4',
 'Q8NBT2',
 'A0FGR8',
 'O60645',
 'Q9UNH6',
 'Q9UNH7',
 'Q8WWI1',
 'Q8NBJ7',
 'Q8NCA5',
 'Q9UJ41',
 'Q8IY17',
 'P11586',
 'Q99729',
 'Q9NRG7',
 'Q9UM54',
 'Q13459',
 'Q86TG7',
 'P18887']

In [23]:
# for R figures
os.chdir("/Users/mariapalafox/Box Sync/CODE_DATA/dir_MAPpaper/R_figures")

In [10]:
# importing files that, once dynamic slope scores are added to grouped by ID version of files, will be labeled
false_labeled = pd.read_csv("Rmerge_MISMAP_SCORED_falseIdentity_1805UKBidsX5_9025rows_allReleases_slope0_dynamic_colnames.csv")
all_labeled = pd.read_csv("Rmerge_MISMAP_SCORED_allIdentity_3979UKBidsX5_19895rows_allReleases_slope0_dynamic_colnames.csv")
describeMe(all_labeled)
describeMe(false_labeled)

# pull out these IDs in uniprot File
ukbfasta = pd.read_csv("/Users/mariapalafox/Box Sync/CODE_DATA/dir_MAPpaper/TSV_UNIPROT_xref/MISMAP_SCORE_UniprotFastaCKabund_w_UniprotCYSLYSpositionsLabeled_3979.csv")
ukbfasta.drop(['Unnamed: 0'],inplace=True,axis=1)
ukbfasta.head(10)

# list of uniprot IDs with no canonical sequence match in any of the ensembl releases
shared = ['P07686', 'Q9P2N6', 'P11532', 'Q9BX63', 'Q8WX93', 'Q68E01', 'Q96ME1', 'O15392', 'Q9BV68', 'O75400', 'Q14135', 'Q14558', 'Q6PKG0', 'O43708', 'Q03001', 'P16278', 'O14965', 'Q15170', 'Q9Y679', 'Q9Y520', 'Q12912', 'P36639', 'P30837', 'P02765', 'P53990', 'A6NNF4', 'Q9BZ29', 'O15061', 'P20839', 'P17927', 'P11182', 'Q9UMY4', 'Q8NBT2', 'A0FGR8', 'O60645', 'Q9UNH6', 'Q9UNH7', 'Q8WWI1', 'Q8NBJ7', 'Q8NCA5', 'Q9UJ41', 'Q8IY17', 'P11586', 'Q99729', 'Q9NRG7', 'Q9UM54', 'Q13459', 'Q86TG7', 'P18887']
print(len(shared))

# adding column based on whether ID is one of 49 uniprot IDs that have no ensp canoncical seq match
ukb49 = addcolumnconditionalDrop(shared, ukbfasta, "UniProtSP_xref", "noCanonicalENSP49")
describeMe(ukb49)
all_labeled49 = addcolumnconditional(shared, all_labeled, "UniProtID", "noCanonicalENSP49")
checkColumnValues(all_labeled49, "noCanonicalENSP49")
false_labeled49 = addcolumnconditional(shared, false_labeled, "UniProtID", "noCanonicalENSP49")
checkColumnValues(false_labeled49, "noCanonicalENSP49")

(19895, 9)
Index(['UniProtID', 'total_targets', 'found_count', 'missed_count',
       'frac_found', 'frac_missed', 'release', 'notSlope0', 'dynamicSlope'],
      dtype='object')
UniProtID        0
total_targets    0
found_count      0
missed_count     0
frac_found       0
frac_missed      0
release          0
notSlope0        0
dynamicSlope     0
dtype: int64
(9025, 9)
Index(['UniProtID', 'total_targets', 'found_count', 'missed_count',
       'frac_found', 'frac_missed', 'release', 'notSlope0', 'dynamicSlope'],
      dtype='object')
UniProtID        0
total_targets    0
found_count      0
missed_count     0
frac_found       0
frac_missed      0
release          0
notSlope0        0
dynamicSlope     0
dtype: int64
49
(49, 9)
Index(['UniProtSP_xref', 'UniProt_length', 'proSequence', 'C_abundance',
       'K_abundance', 'in3979xref', 'total_targets', 'pos_dict',
       'noCanonicalENSP49'],
      dtype='object')
UniProtSP_xref       0
UniProt_length       0
proSequence          0
C_abunda

In [11]:
colnames = ['UniProtID', 'total_targets', 'found_count', 'missed_count',
       'frac_found', 'frac_missed', 'release', 'notSlope0', 'dynamicSlope4',
       'noCanonicalENSP49']
all_labeled49.columns = colnames
false_labeled49.columns=colnames

In [12]:
# importing files with grouped by uniprot ID in order to assess how frac missed scores compare in all releases
allMISSED = pd.read_csv("/Users/mariapalafox/Box Sync/CODE_DATA/dir_MAPpaper/R_figures/R_allIdentity_fracMISSED_slope0_dynamic_colnames_3979.csv")
falseMISSED = pd.read_csv("/Users/mariapalafox/Box Sync/CODE_DATA/dir_MAPpaper/R_figures/R_falseOnlyIdentity_fracMISSED_slope0_dynamic_colnames_1805.csv")

In [13]:
# perviously used to check slope 0 and dynmic slope - 4 values out of 5 are unique
# expanding this out to include dynamic levels - 1 2 3 
def mismaplines_dynamic(df):
    diffline = []
    for index, row in df.iterrows():
        ukbid = row['UniProtID']
        ls = row['frac_missed']
        python_ls = literal_eval(ls) 
        lenLS = len(set(python_ls))
        if lenLS == 5:
            diffline.append("5")
        if lenLS == 4:
            diffline.append("4")
        if lenLS == 3:
            diffline.append("3")
        if lenLS == 2:
            diffline.append("2")
        if lenLS == 1:
            diffline.append("1")
    df.loc[:,'dynamic_slope_scores'] = diffline
    print(df.shape)

In [14]:
mismaplines_dynamic(allMISSED)
checkColumnValues(allMISSED, 'dynamic_slope_scores')
describeMe(allMISSED)

mismaplines_dynamic(falseMISSED)
checkColumnValues(falseMISSED, 'dynamic_slope_scores')
describeMe(falseMISSED)


allMISSED49 = addcolumnconditional(shared, allMISSED, "UniProtID", "noCanonicalENSP49")
checkColumnValues(allMISSED49, "noCanonicalENSP49")
describeMe(allMISSED49)


falseMISSED49 = addcolumnconditional(shared, falseMISSED, "UniProtID", "noCanonicalENSP49")
checkColumnValues(falseMISSED49, "noCanonicalENSP49")
describeMe(falseMISSED49)

(3979, 5)
  dynamic_slope_scores  Count
0                    1   3071
1                    2    854
2                    3     43
3                    4     11
(3979, 5)
Index(['UniProtID', 'frac_missed', 'slope0', 'dynamic_slope',
       'dynamic_slope_scores'],
      dtype='object')
UniProtID               0
frac_missed             0
slope0                  0
dynamic_slope           0
dynamic_slope_scores    0
dtype: int64
(1805, 5)
  dynamic_slope_scores  Count
0                    1   1523
1                    2    269
2                    3     12
3                    4      1
(1805, 5)
Index(['UniProtID', 'frac_missed', 'slope0', 'dynamic_slope',
       'dynamic_slope_scores'],
      dtype='object')
UniProtID               0
frac_missed             0
slope0                  0
dynamic_slope           0
dynamic_slope_scores    0
dtype: int64
  noCanonicalENSP49  Count
0             False   3930
1              True     49
(3979, 6)
Index(['UniProtID', 'frac_missed', 'slope0', 'dynam

In [15]:
def dropColumn(df, colname):
    df.drop([colname],inplace=True, axis=1)
    
dropColumn(allMISSED49, 'slope0')
dropColumn(allMISSED49, 'dynamic_slope')
dropColumn(falseMISSED49, 'slope0')
dropColumn(falseMISSED49, 'dynamic_slope')

In [16]:
allMISSED49.head(5)

,UniProtID,frac_missed,dynamic_slope_scores,noCanonicalENSP49
0,A0AVT1,"[0.4090909090909091, 0.4090909090909091, 0.409...",1,False
1,A0FGR8,"[1.0, 1.0, 1.0, 1.0, 1.0]",1,True
2,A0JNW5,"[0.6666666666666666, 0.5, 0.5, 0.5, 0.5]",2,False
3,A0MZ66,"[0.5, 0.8, 0.8, 0.8, 0.8]",2,False
4,A1KXE4,"[0.0, 0.0, 0.0, 0.0, 0.0]",1,False


In [20]:
all_labeled49.head(3)

,UniProtID,total_targets,found_count,missed_count,frac_found,frac_missed,release,notSlope0,dynamicSlope4,noCanonicalENSP49
0,A0AVT1,22,13,9,0.590909,0.409091,85,False,False,False
1,A0FGR8,1,0,1,0.000000,1.000000,85,False,False,True
2,A0JNW5,3,1,2,0.333333,0.666667,85,True,False,False


In [17]:
falseMISSED49.head(4)

,UniProtID,frac_missed,dynamic_slope_scores,noCanonicalENSP49
0,A0AVT1,"[0.8181818181818182, 0.8181818181818182, 0.818...",1,False
1,A0FGR8,"[1.0, 1.0, 1.0, 1.0, 1.0]",1,True
2,A0JNW5,"[1.0, 1.0, 1.0, 1.0, 1.0]",1,False
3,A0MZ66,"[0.6666666666666666, 1.0, 1.0, 1.0, 1.0]",2,False


In [21]:
false_labeled49.head(4)

,UniProtID,total_targets,found_count,missed_count,frac_found,frac_missed,release,notSlope0,dynamicSlope4,noCanonicalENSP49
0,A0AVT1,11,2,9,0.181818,0.818182,85,False,False,False
1,A0FGR8,1,0,1,0.000000,1.000000,85,False,False,True
2,A0JNW5,2,0,2,0.000000,1.000000,85,False,False,False
3,A0MZ66,3,1,2,0.333333,0.666667,85,True,False,False


In [24]:
dropColumn(all_labeled49, 'notSlope0')
dropColumn(all_labeled49, 'dynamicSlope4')
dropColumn(false_labeled49, 'notSlope0')
dropColumn(false_labeled49, 'dynamicSlope4')

In [22]:
# saving files that will be used to label all release merge file, 3979x5 and 1805x5, with dynamic slope scores
allMISSED49.to_csv("/Users/mariapalafox/Box Sync/CODE_DATA/dir_MAPpaper/R_figures/R_allIdentity_fracMISSED_dynamic_slope_scores_noCanonical49_3979.csv",index=False) 

falseMISSED49.to_csv("/Users/mariapalafox/Box Sync/CODE_DATA/dir_MAPpaper/R_figures/R_falseIdentity_fracMISSED_dynamic_slope_scores_noCanonical49_1805.csv",index=False) 

#ukb49.to_csv("MISMAP_SCORE_UniprotFastaCKabund_w_UniprotCYSLYSpositionsLabeled_49.csv",index=False)

# using the dynamic slope scores column added to groupedby UKBID datasets to label each UKB ID in file not grouped by UKBID (length is either 3979 x 5 or 1805 x 5...each release has score for each UKBID)

In [26]:
# creating a dictionary of uniprotID:dynamic_slope_scores in order to map these values to the all_labeled49 df
# in R figure its best to have 1 column with dynamic score values instead of 4 columns with True False for scores
ref_slope = dict(zip(allMISSED49.UniProtID, allMISSED49.dynamic_slope_scores))
false_ref_slope = dict(zip(falseMISSED49.UniProtID, falseMISSED49.dynamic_slope_scores))

In [27]:
# duplicating uniprotID column inorder to convert these IDs into there dynamicslopescores with dictionary
all_labeled49['dynamic_slope_scores'] = all_labeled49['UniProtID']
false_labeled49['dynamic_slope_scores'] = false_labeled49['UniProtID']

all_labeled49.dynamic_slope_scores = all_labeled49.dynamic_slope_scores.map(ref_slope)
false_labeled49.dynamic_slope_scores = false_labeled49.dynamic_slope_scores.map(false_ref_slope)

In [28]:
all_labeled49.head(3)

,UniProtID,total_targets,found_count,missed_count,frac_found,frac_missed,release,noCanonicalENSP49,dynamic_slope_scores
0,A0AVT1,22,13,9,0.590909,0.409091,85,False,1
1,A0FGR8,1,0,1,0.000000,1.000000,85,True,1
2,A0JNW5,3,1,2,0.333333,0.666667,85,False,2


In [29]:
false_labeled49.head(3)

,UniProtID,total_targets,found_count,missed_count,frac_found,frac_missed,release,noCanonicalENSP49,dynamic_slope_scores
0,A0AVT1,11,2,9,0.181818,0.818182,85,False,1
1,A0FGR8,1,0,1,0.000000,1.000000,85,True,1
2,A0JNW5,2,0,2,0.000000,1.000000,85,False,1


---
---
---
---
---



# calculating absolute slope values for each uniprot ID

In [39]:
ukbfasta = pd.read_csv("/Users/mariapalafox/Box Sync/CODE_DATA/dir_MAPpaper/TSV_UNIPROT_xref/MISMAP_SCORE_UniprotFastaCKabund_w_UniprotCYSLYSpositionsLabeled_3979.csv")
ukbfasta.drop(['Unnamed: 0'],inplace=True,axis=1)

In [57]:
ukbfasta.head(4)

,UniProtSP_xref,UniProt_length,proSequence,C_abundance,K_abundance,in3979xref,total_targets,pos_dict
0,Q9HAS0,396,MLPSLQESMDGDEKELESSEEGGSAEERRLEPPSSSHYCLYSYRGS...,0.0303,0.0581,True,1,{182: 'C'}
1,Q86X76,327,MLGFITRPPHRFLSLLCPGLRIPQLSVLCAQPRPRAMAISSSSCEL...,0.0459,0.0245,True,3,"{161: 'K', 165: 'C', 203: 'C'}"
2,Q9NQR4,276,MTSFRLALIQLQISSIKSDNVTRACSFIREAATQGAKIVSLPECFN...,0.0254,0.0652,True,7,"{44: 'C', 52: 'K', 123: 'K', 130: 'K', 146: 'C..."
3,Q9Y314,301,MTRHGKNCTAGAVYTYHEKKKDTAASGYGTQNIRLSRDAVKDFDCC...,0.0299,0.0797,True,3,"{8: 'C', 78: 'K', 185: 'C'}"


In [62]:
# create dict of uniprotID and number of targets assocaited then add this to false_labeled and all_labeled
ref_targets = dict(zip(ukbfasta.UniProtSP_xref, ukbfasta.total_targets))
ref_cys = dict(zip(ukbfasta.UniProtSP_xref, ukbfasta.C_abundance))
ref_lys = dict(zip(ukbfasta.UniProtSP_xref, ukbfasta.K_abundance))
ref_length = dict(zip(ukbfasta.UniProtSP_xref, ukbfasta.UniProt_length))

len(ref_targets)

3979

In [44]:
# adding total targets column to each file
all_labeled49['total_targets_ukbID'] = all_labeled49['UniProtID']
false_labeled49['total_targets_ukbID'] = false_labeled49['UniProtID']

all_labeled49.total_targets_ukbID = all_labeled49.total_targets_ukbID.map(ref_targets)
false_labeled49.total_targets_ukbID = false_labeled49.total_targets_ukbID.map(ref_targets)

In [63]:
all_labeled49['UKB_Cys_abundance'] = all_labeled49['UniProtID']
false_labeled49['UKB_Cys_abundance'] = false_labeled49['UniProtID']

all_labeled49.UKB_Cys_abundance = all_labeled49.UKB_Cys_abundance.map(ref_cys)
false_labeled49.UKB_Cys_abundance = false_labeled49.UKB_Cys_abundance.map(ref_cys)

In [64]:
all_labeled49['UKB_Lys_abundance'] = all_labeled49['UniProtID']
false_labeled49['UKB_Lys_abundance'] = false_labeled49['UniProtID']

all_labeled49.UKB_Lys_abundance = all_labeled49.UKB_Lys_abundance.map(ref_lys)
false_labeled49.UKB_Lys_abundance = false_labeled49.UKB_Lys_abundance.map(ref_lys)

In [65]:
all_labeled49['UKB_length'] = all_labeled49['UniProtID']
false_labeled49['UKB_length'] = false_labeled49['UniProtID']

all_labeled49.UKB_length = all_labeled49.UKB_length.map(ref_length)
false_labeled49.UKB_length = false_labeled49.UKB_length.map(ref_length)

In [66]:
ukbfasta[ukbfasta['UniProtSP_xref'] == 'P08397']

,UniProtSP_xref,UniProt_length,proSequence,C_abundance,K_abundance,in3979xref,total_targets,pos_dict
3693,P08397,361,MSGNGNAAATAEENSPKMRVIRVGTRKSQLARIQTDSVVATLKASY...,0.0111,0.0526,True,5,"{70: 'K', 74: 'K', 87: 'K', 98: 'K', 261: 'C'}"


In [48]:
# adding number of ENSP IDs linked to each UKB ID for each release
all_labeled49['ENSPID_count'] = all_labeled49['total_targets'] / all_labeled49['total_targets_ukbID']
false_labeled49['ENSPID_count'] = false_labeled49['total_targets'] / false_labeled49['total_targets_ukbID']

In [51]:
false_labeled49['ENSPID_count'] = false_labeled49['ENSPID_count'].astype(int)
all_labeled49['ENSPID_count'] = all_labeled49['ENSPID_count'].astype(int)

In [67]:
false_labeled49.dtypes

UniProtID                object
total_targets             int64
found_count               int64
missed_count              int64
frac_found              float64
frac_missed             float64
release                   int64
noCanonicalENSP49        object
dynamic_slope_scores     object
total_targets_ukbID       int64
ENSPID_count              int64
UKB_Cys_abundance       float64
UKB_Lys_abundance       float64
UKB_length                int64
dtype: object

In [68]:
false_labeled49[false_labeled49['UniProtID'] == 'P08397']

,UniProtID,total_targets,found_count,missed_count,frac_found,frac_missed,release,noCanonicalENSP49,dynamic_slope_scores,total_targets_ukbID,ENSPID_count,UKB_Cys_abundance,UKB_Lys_abundance,UKB_length
282,P08397,50,24,26,0.480000,0.520000,85,False,3,5,10,0.0111,0.0526,361
2087,P08397,50,24,26,0.480000,0.520000,92,False,3,5,10,0.0111,0.0526,361
3892,P08397,50,20,30,0.400000,0.600000,94,False,3,5,10,0.0111,0.0526,361
5697,P08397,60,28,32,0.466667,0.533333,96,False,3,5,12,0.0111,0.0526,361
7502,P08397,50,24,26,0.480000,0.520000,97,False,3,5,10,0.0111,0.0526,361


In [69]:
all_labeled49[all_labeled49['UniProtID'] == 'P08397']

,UniProtID,total_targets,found_count,missed_count,frac_found,frac_missed,release,noCanonicalENSP49,dynamic_slope_scores,total_targets_ukbID,ENSPID_count,UKB_Cys_abundance,UKB_Lys_abundance,UKB_length
599,P08397,60,34,26,0.566667,0.433333,85,False,4,5,12,0.0111,0.0526,361
4578,P08397,60,34,26,0.566667,0.433333,92,False,4,5,12,0.0111,0.0526,361
8557,P08397,60,30,30,0.500000,0.500000,94,False,4,5,12,0.0111,0.0526,361
12536,P08397,70,38,32,0.542857,0.457143,96,False,4,5,14,0.0111,0.0526,361
16515,P08397,50,24,26,0.480000,0.520000,97,False,4,5,10,0.0111,0.0526,361


In [74]:
allMISSED49[allMISSED49['UniProtID'] == 'P08397']

,UniProtID,frac_missed,dynamic_slope_scores,noCanonicalENSP49
599,P08397,"[0.43333333333333335, 0.43333333333333335, 0.5...",4,False


In [70]:
all_labeled49.corr()

,total_targets,found_count,missed_count,frac_found,frac_missed,release,total_targets_ukbID,ENSPID_count,UKB_Cys_abundance,UKB_Lys_abundance,UKB_length
total_targets,1.000000,0.865903,0.708705,-0.141064,0.141064,5.013647e-03,6.351875e-01,0.472420,-3.192365e-02,7.838295e-02,6.725828e-02
found_count,0.865903,1.000000,0.260768,0.043594,-0.043594,3.147960e-03,5.744430e-01,0.432967,-2.256935e-02,9.518825e-02,1.078639e-02
missed_count,0.708705,0.260768,1.000000,-0.333736,0.333736,5.236346e-03,4.157014e-01,0.301102,-2.978009e-02,1.702331e-02,1.145943e-01
frac_found,-0.141064,0.043594,-0.333736,1.000000,-1.000000,-2.797279e-02,-7.931625e-03,-0.346319,2.802001e-02,5.065630e-02,-1.234778e-01
frac_missed,0.141064,-0.043594,0.333736,-1.000000,1.000000,2.797279e-02,7.931625e-03,0.346319,-2.802001e-02,-5.065630e-02,1.234778e-01
release,0.005014,0.003148,0.005236,-0.027973,0.027973,1.000000e+00,-1.633079e-15,0.016904,-1.375888e-17,1.947252e-17,-9.600398e-17
total_targets_ukbID,0.635188,0.574443,0.415701,-0.007932,0.007932,-1.633079e-15,1.000000e+00,0.016968,-4.959340e-02,1.576686e-01,8.145346e-02
ENSPID_count,0.472420,0.432967,0.301102,-0.346319,0.346319,1.690406e-02,1.696840e-02,1.000000,-1.110456e-02,-1.742072e-02,4.729734e-02
UKB_Cys_abundance,-0.031924,-0.022569,-0.029780,0.028020,-0.028020,-1.375888e-17,-4.959340e-02,-0.011105,1.000000e+00,-6.440400e-02,-4.908852e-02
UKB_Lys_abundance,0.078383,0.095188,0.017023,0.050656,-0.050656,1.947252e-17,1.576686e-01,-0.017421,-6.440400e-02,1.000000e+00,-5.508702e-02


In [71]:
false_labeled49.corr()

,total_targets,found_count,missed_count,frac_found,frac_missed,release,total_targets_ukbID,ENSPID_count,UKB_Cys_abundance,UKB_Lys_abundance,UKB_length
total_targets,1.000000,0.589534,0.922365,-0.034819,0.034819,-6.717133e-03,6.485974e-01,0.387824,-4.805892e-02,5.894543e-02,9.867727e-02
found_count,0.589534,1.000000,0.231719,0.272078,-0.272078,-5.211934e-03,4.856513e-01,0.269402,-3.762976e-02,4.878997e-02,3.479652e-02
missed_count,0.922365,0.231719,1.000000,-0.172060,0.172060,-5.596877e-03,5.488475e-01,0.338218,-3.988121e-02,4.765448e-02,1.021970e-01
frac_found,-0.034819,0.272078,-0.172060,1.000000,-1.000000,-3.690150e-03,-4.830037e-02,-0.040028,8.135459e-03,-6.258037e-03,-1.676049e-02
frac_missed,0.034819,-0.272078,0.172060,-1.000000,1.000000,3.690150e-03,4.830037e-02,0.040028,-8.135459e-03,6.258037e-03,1.676049e-02
release,-0.006717,-0.005212,-0.005597,-0.003690,0.003690,1.000000e+00,1.876780e-16,-0.002500,-1.287895e-17,-5.785106e-18,1.877596e-18
total_targets_ukbID,0.648597,0.485651,0.548847,-0.048300,0.048300,1.876780e-16,1.000000e+00,-0.003477,-6.349026e-02,1.427561e-01,9.333734e-02
ENSPID_count,0.387824,0.269402,0.338218,-0.040028,0.040028,-2.500407e-03,-3.477214e-03,1.000000,-2.951647e-02,-2.892333e-02,7.535225e-02
UKB_Cys_abundance,-0.048059,-0.037630,-0.039881,0.008135,-0.008135,-1.287895e-17,-6.349026e-02,-0.029516,1.000000e+00,-6.344266e-02,-5.974064e-02
UKB_Lys_abundance,0.058945,0.048790,0.047654,-0.006258,0.006258,-5.785106e-18,1.427561e-01,-0.028923,-6.344266e-02,1.000000e+00,7.377017e-03


In [73]:
# saving files that will be loaded into R to make laser figures
all_labeled49.to_csv("/Users/mariapalafox/Box Sync/CODE_DATA/dir_MAPpaper/R_figures/Rmerge_MISMAP_SCORED_allIdentity_3979UKBidsX5_19895rows_allReleases_49noCanonicalMatch_dynamicSlopeScores_colnames.csv",index=False)
false_labeled49.to_csv("/Users/mariapalafox/Box Sync/CODE_DATA/dir_MAPpaper/R_figures/Rmerge_MISMAP_SCORED_falseIdentity_1805UKBidsX5_9025rows_allReleases_49noCanonicalMatch_dynamicSlopeScores_colnames.csv",index=False)